<a href="https://colab.research.google.com/github/dsporcaro/Anythink-Market-73vwk/blob/main/Learning_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Code for data collection

##Installations needed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!pip install neo4j tqdm

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
!pip install BeautifulSoup4

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.wSgPoBNv9e/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.OSMCQjimkZ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.nKSXDCD8We/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [ ]:
!pip install selenium

## Credentials

In [ ]:
import requests
from neo4j import GraphDatabase

# David's Dev environment
NEO4J_URI = "neo4j+s://2a5877d3.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "1WdDxFnlqo8l-EjYez4FJVFqenJAR6G4b21A0HNlbFQ"

# Production
#NEO4J_URI = "neo4j://db.learningdesignalliance.org:7687"
#NEO4J_USER = "neo4j"
#NEO4J_PASSWORD = "F8x5s3R6t7TdW7c"

##Common Standards

###Extract

In [ ]:
import requests
import json
import tqdm

# Your API key
headers = {'Api-Key': commonStandards_api_key}
base_url = 'http://commonstandardsproject.com/api/v1/'

# Fetch Jurisdictions
jurisdiction_response = requests.get(f'{base_url}jurisdictions/', headers=headers)
jurisdictions = jurisdiction_response.json()['data']

# Target standard set indexes
target_indexes = [37, 113]
current_index = 0  # Initialize a counter for the standard sets

# Loop through Jurisdictions to get Standard Set IDs
with open("standard_set.jsonl", "w", encoding="utf-8") as file:  # Use "w" to overwrite existing or create new file
    for jurisdiction in tqdm.tqdm(jurisdictions):
        jurisdiction_id = jurisdiction['id']
        standard_sets_response = requests.get(f'{base_url}/jurisdictions/{jurisdiction_id}', headers=headers)

        for standard_set in standard_sets_response.json()['data']['standardSets']:
            current_index += 1  # Increment the counter for each standard set
            if current_index in target_indexes:  # Check if the current index is one of the targets
                standard_set_id = standard_set['id']
                try:
                    detailed_standard_set_response = requests.get(f'{base_url}standard_sets/{standard_set_id}', headers=headers)
                    if detailed_standard_set_response.status_code == 200:
                        detailed_standard_set = detailed_standard_set_response.json()
                        file.write(json.dumps(detailed_standard_set) + "\n")
                        print(f"Fetched and stored detailed standard set: {detailed_standard_set}")
                    else:
                        print(f"Failed to fetch details for standard set ID {standard_set_id}, Status code: {detailed_standard_set_response.status_code}")
                except Exception as e:
                    print(f"Error occurred while fetching detailed standard set: {e}")

# Read back the collected standard sets for verification or further processing
with open("standard_set.jsonl", "r", encoding="utf-8") as file:
    detailed_standard_sets = [json.loads(line) for line in file if line.strip()]

print(f"Collected {len(detailed_standard_sets)} detailed standard sets.")


  2%|▏         | 10/602 [00:02<03:10,  3.10it/s]

Fetched and stored detailed standard set: {'data': {'id': 'C6C2A7C25C90437AB98689888FDEE09A', 'title': 'AP Calculus AB & BC (2016)', 'subject': 'AP Calculus', 'educationLevels': [], 'cspStatus': {}, 'license': {'title': 'CC BY 4.0 US', 'URL': 'http://creativecommons.org/licenses/by/4.0/us/', 'rightsHolder': 'Common Curriculum, Inc.'}, 'document': {'title': 'AP Calculus AB and BC Course and Exam Description', 'sourceURL': 'https://secure-media.collegeboard.org/digitalServices/pdf/ap/ap-calculus-ab-and-bc-course-and-exam-description.pdf'}, 'jurisdiction': {'id': '0A5FD99233A74D8FA3A74F52E5F6CDEC', 'title': 'College Board'}, 'standards': {'A070C3570D7542F98E5F6D25CD355D5E': {'id': 'A070C3570D7542F98E5F6D25CD355D5E', 'asnIdentifier': None, 'position': 56000, 'depth': 2, 'listId': 'C', 'statementNotation': '4.2C', 'altStatementNotation': None, 'statementLabel': None, 'description': 'Determine the radius and interval of convergence of a power series.', 'comments': [], 'ancestorIds': ['F47233

  2%|▏         | 11/602 [00:03<03:43,  2.65it/s]

Fetched and stored detailed standard set: {'data': {'id': 'CF057E3B37F9477D851FE33DEE8E85F1', 'title': 'Biology', 'subject': 'Science', 'educationLevels': ['10'], 'cspStatus': {}, 'license': {'title': 'CC BY 4.0 US', 'URL': 'http://creativecommons.org/licenses/by/4.0/us/', 'rightsHolder': 'Common Curriculum, Inc.'}, 'document': {'sourceURL': 'https://www.aqa.org.uk/subjects/science/gcse', 'title': 'AQA'}, 'jurisdiction': {'id': '3A4F64A6B79845888BADD3F0EC2CFF66', 'title': 'AQA (UK)'}, 'standards': {'5C486D3457A94E899E2FEFB6ED5263EF': {'id': '5C486D3457A94E899E2FEFB6ED5263EF', 'asnIdentifier': None, 'position': 128000, 'depth': 1, 'listId': 'H', 'statementNotation': '8', 'altStatementNotation': None, 'statementLabel': None, 'description': 'Key ideas', 'comments': [], 'ancestorIds': ['F8C70CB68B2748D1AFF74BEE8B33C8D2']}, '7B02B7A8C91C48B0A44A015D047305E7': {'id': '7B02B7A8C91C48B0A44A015D047305E7', 'asnIdentifier': None, 'position': 127000, 'depth': 3, 'listId': 'd', 'statementNotation':

100%|██████████| 602/602 [02:13<00:00,  4.51it/s]

Collected 2 detailed standard sets.


###Load and Transform

In [ ]:
import json
from neo4j import GraphDatabase

# Neo4j database connection setup
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def process_standard_set(tx, detailed_standard_set):
    if not isinstance(detailed_standard_set, dict):
        print("Skipping invalid detailed_standard_set")
        return

    data = detailed_standard_set.get('data', {})
    jurisdiction_originalId = data.get('jurisdiction', {}).get('id')
    jurisdiction_name = data.get('jurisdiction', {}).get('title')
    standard_originalId = data.get('id')
    standard_name = data.get('title')
    subject_name = data.get('subject')
    education_level = ", ".join(data.get('educationLevels', []))
    license_name = data.get('license', {}).get('title', 'Unknown License')
    licenseURL = data.get('license', {}).get('URL', 'No URL Provided')
    rightsHolder = data.get('license', {}).get('rightsHolder', 'Unknown RightsHolder')
    standard_date = data.get('document', {}).get('valid', '')
    standard_sourceURL = data.get('document', {}).get('sourceURL', 'No URL Provided')
    standard_status = data.get('document', {}).get('publicationStatus', 'Unknown Status')

    # Create Jurisdiction
    tx.run("""
        MERGE (j:Jurisdiction {originalId: $originalId})
        ON CREATE SET j.name = $name
    """, originalId=jurisdiction_originalId, name=jurisdiction_name)

    # Create Subject
    tx.run("""
        MERGE (sub:Subject {name: $name})
    """, name=subject_name)

    # Create Standard node and link to Jurisdiction
    tx.run("""
        MERGE (std:Standard {originalId: $originalId})
        ON CREATE SET std.name = $name,
                      std.educationLevel = $educationLevel,
                      std.sourceURL = $sourceURL,
                      std.license = $license,
                      std.licenseURL = $licenseURL,
                      std.rightsHolder = $rightsHolder,
                      std.date = $date,
                      std.status = $status
        WITH std
        MATCH (j:Jurisdiction {originalId: $jurisdictionId})
        MERGE (j)-[:PUBLISHES]->(std)
    """, originalId=standard_originalId, name=standard_name, educationLevel=education_level, sourceURL=standard_sourceURL, license=license_name, licenseURL=licenseURL, rightsHolder=rightsHolder, date=standard_date, status=standard_status, jurisdictionId=jurisdiction_originalId)

    # Link Standard to Subject
    tx.run("""
        MATCH (std:Standard {originalId: $originalId}), (sub:Subject {name: $subjectName})
        MERGE (std)-[:ASSOCIATED_WITH]->(sub)
    """, originalId=standard_originalId, subjectName=subject_name)

    # Preparing data for batch processing
    standards_list = [{
        "originalId": standard_id,
        "name": standard.get("description"),
        "depth": standard.get("depth"),
        "notation": standard.get("statementNotation", ""),
        "ancestorIds": standard.get("ancestorIds", [])
    } for standard_id, standard in data.get('standards', {}).items()]

    tx.run("""
        UNWIND $standards AS standard
        FOREACH(ignoreMe IN CASE WHEN standard.depth <= 1 THEN [1] ELSE [] END |
            MERGE (comp:Competency {originalId: standard.originalId})
            ON CREATE SET comp.name = standard.name, comp.depth = standard.depth, comp.notation = standard.notation
        )
        FOREACH(ignoreMe IN CASE WHEN standard.depth >= 2 THEN [1] ELSE [] END |
            MERGE (skill:Skill {originalId: standard.originalId})
            ON CREATE SET skill.name = standard.name, skill.depth = standard.depth, skill.notation = standard.notation
            )
    """, parameters={"standards": standards_list})

    # Create DESCRIBES relationships for standards without ancestors
    describes_list = [s for s in standards_list if not s['ancestorIds']]
    tx.run("""
        UNWIND $describes AS desc
        MATCH (std:Standard {originalId: $standardOriginalId}), (node) WHERE node.originalId = desc.originalId AND (node:Competency OR node:Skill)
        MERGE (std)-[:DESCRIBES]->(node)
    """, parameters={"describes": describes_list, "standardOriginalId": standard_originalId})

    # Prepare data for HAS_CHILD relationships considering only the immediate parent
    has_child_list = []
    for item in standards_list:
        if item["ancestorIds"]:
            immediate_parent_id = item["ancestorIds"][0]  # Adjusted to use the first ancestorId
            has_child_list.append({"childId": item["originalId"], "parentId": immediate_parent_id})

    # batch create HAS_CHILD relationships
    tx.run("""
        UNWIND $pairs AS pair
        MATCH (parent) WHERE parent.originalId = pair.parentId AND (parent:Competency OR parent:Skill)
        MATCH (child) WHERE child.originalId = pair.childId AND (child:Competency OR child:Skill)
        MERGE (parent)-[:HAS_CHILD]->(child)
    """, parameters={"pairs": has_child_list})

with driver.session() as session:
    with open("standard_set.jsonl", "r", encoding="utf-8") as file:
        for line in file:
            detailed_standard_set = json.loads(line)
            process_standard_set(session, detailed_standard_set)

driver.close()


##Open Skills

###Extract

In [ ]:
def get_data():
    url = "https://www.openrsd.com/api/search2"
    batch_size = 100
    total_count = 17520

    with open("openrsd_links.txt", "w+") as file:
        file.write("")

    for i in tqdm.tqdm(range(0, total_count, batch_size)):
        params = url + "?start=" + str(i) + "&length=" + str(batch_size)

        response = requests.get(params)
        if response.status_code == 200:
            data = response.json()
            results = data['results']
            print(f"Processing {len(results)} results")
            for result in results:
                try:
                    if result["attachmentCount"] > 0:
                        attachments = result['attachments']
                        print(f"Found {len(attachments)} attachments")
                        for attachment in attachments:
                            view = attachment['links']['view']
                            with open("openrsd_links_2.txt", "a+") as file:
                                file.write(view + "\n")
                except Exception as e:
                    print(e)

        else:
            print(f"Failed to fetch data for {params}, Status code: {response.status_code}")

def process_data():
    with open("openrsd_links_2.txt", "r+") as file:
        links = file.read().split("\n")

    def process_open_standards(tx, standard):
        # Preparing data for UNWIND
        skills_data = [{
            "skillName": skill.get("skillName", "Unknown Skill"),
            "skillStatement": skill.get("skillStatement", ""),
            "category": skill.get("category", "Unknown Category")
        } for skill in standard.get("skills", [])]

        # Merge Jurisdiction and Standard, then UNWIND skills for batch processing
        tx.run("""
        MERGE (jur:Jurisdiction {name: $author})
        MERGE (std:Standard {url: $url, title: $name, status: $status,
                             publishDate: $publishDate, creationDate: $creationDate})
        MERGE (jur)-[:HAS_STANDARD]->(std)
        WITH std
        UNWIND $skills_data AS skill
            MERGE (comp:Competency {title: skill.category})
            MERGE (skl:Skill {name: skill.skillName, description: skill.skillStatement})
            MERGE (std)-[:INCLUDES]->(comp)
            MERGE (comp)-[:HAS_SKILL]->(skl)
        """, author=standard.get("author", "Unknown Author"), url=standard.get("id", ""),
               name=standard.get("name", ""), status=standard.get("status", "Unknown Status"),
               publishDate=standard.get("publishDate", ""), creationDate=standard.get("creationDate", ""),
               skills_data=skills_data)

    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    with driver.session() as session:
        for link in tqdm(links):
            if link:
                response = requests.get(link, allow_redirects=True)
                if response.url:
                    link = response.url

                response = requests.get(link)

                if response.status_code == 200:
                    data = response.json()
                    print(len(data))
                    process_open_standards(session, data)
                    print(data)
                else:
                    print(f"Failed to fetch data for {link}, Status code: {response.status_code}")

get_data()
process_data()

NameError: name 'tqdm' is not defined

In [ ]:
print(open_standards_data[0])

{'type': 'RichSkillCollection', 'creationDate': '2022-11-28T22:15:55.95894Z', 'name': 'Accountants and Auditors_Dec.2022', 'id': 'https://osmt.wgu.edu/api/collections/1db79be3-ff33-4114-8449-e151aa9e6b25', 'owner': '', 'status': 'published', 'description': 'This Collection represents the skills necessary for professionals responsible for all preparation and examination of financial documents, reports, or budgets, including advising others on business, financial, or operations matters.  It covers expectations for the CIA, CMA, CFE, and CPA certifications and includes knowledge based on the National Initiative for Cybersecurity Education (NICE) standards. This collection of skills was developed in partnership with a panel of subject matter experts who have worked as Accountants and Auditors.', 'creator': 'https://credentialengineregistry.org/resources/ce-036d082d-d80e-41a7-99a0-2d63a4ad3a4a', 'skills': [{'id': 'https://osmt.wgu.edu/api/skills/e703c2bc-ac14-441f-aeeb-6a014c054bdd', 'uuid'

complete url list:

In [ ]:
 urls = [
    "https://osmt.wgu.edu/api/collections/1db79be3-ff33-4114-8449-e151aa9e6b25",
    "https://osmt.wgu.edu/api/collections/b24d9ed8-c2bf-4005-a753-a5cf51ddc091",
    "https://osmt.wgu.edu/api/collections/0d5ccab2-14e7-41ed-9588-601423cd7d18",
    "https://osmt.wgu.edu/api/collections/7bfe0550-a69e-42a7-b7f1-b5c23517a516",
    "https://osmt.wgu.edu/api/collections/a730de50-8b9b-4593-8e0e-5b910c8c67a5",
    "https://osmt.wgu.edu/api/collections/8959bdc8-7d46-41e2-91cf-e5abfd2ad09f",
    "https://osmt.wgu.edu/api/collections/701709c6-cdcb-4591-9a69-1a99db829eee",
    "https://osmt.wgu.edu/api/collections/5e11d83b-dacf-4c94-9180-7ff67bd7ef43",
    "https://osmt.wgu.edu/api/collections/8248933b-2a75-46d7-a69b-20d3c7a0dbc6",
    "https://osmt.wgu.edu/api/collections/41f3cbb4-1e70-4766-84e2-a9855c6645de",
    "https://osmt.wgu.edu/api/collections/9cfde8b6-f8c8-471a-a66c-9aabe554fc01",
    "https://osmt.wgu.edu/api/collections/29e22400-5ed7-4d64-9d15-b22748151edc",
    "https://osmt.wgu.edu/api/collections/80be6819-cf5d-44c7-ad5e-3645c7bf8bf0",
    "https://osmt.wgu.edu/api/collections/336e2a53-e979-4a0b-adfa-650684d956ec",
    "https://osmt.wgu.edu/api/collections/b37587df-1911-4ac3-9f6b-3d5bfd355811",
    "https://osmt.wgu.edu/api/collections/920031d9-53ae-4d41-8140-683dc64fa84a",
    "https://osmt.wgu.edu/api/collections/2fe8ca5f-a380-439a-b2a5-9ee5f28b920a",
    "https://osmt.wgu.edu/api/collections/a3669605-3770-48f6-b60f-e3919010c2fc",
    "https://osmt.wgu.edu/api/collections/92e554be-7518-4e2a-ae84-8f41d9f130b3",
    "https://osmt.wgu.edu/api/collections/b48a9c23-08f2-4b27-aa39-1f649881f73a",
    "https://osmt.wgu.edu/api/collections/4beae755-f8dc-4599-b9b6-d4cd44b831a2",
    "https://osmt.wgu.edu/api/collections/f64ef6f9-c345-46de-89e3-fd7e1f3bff48",
    "https://osmt.wgu.edu/api/collections/e162c5ae-c21b-40ff-96c4-6ca83e236333",
    "https://osmt.wgu.edu/api/collections/6de62ad0-4a4c-4076-b3e1-65b991474b09",
    "https://osmt.wgu.edu/api/collections/3797f212-b261-4d92-bc8b-7d604a5276f4",
    "https://osmt.wgu.edu/api/collections/37c8ee1c-0413-4bcf-9f4b-cbbea17adf2d",
    "https://osmt.wgu.edu/api/collections/923f34ed-954e-4c9a-8804-7cbdc31c991b",
    "https://osmt.wgu.edu/api/collections/1ecf7918-b547-440f-8a86-6b7bc37f379e",
    "https://osmt.wgu.edu/api/collections/3104cb5a-4f6b-47a6-97c0-5d046143f60f",
    "https://osmt.wgu.edu/api/collections/c3ebefcb-6e28-49e6-b965-eb8513298384",
    "https://osmt.wgu.edu/api/collections/9be00dcc-d812-42e0-9662-aa5336cf970c",
    "https://osmt.wgu.edu/api/collections/3b9075ec-a1de-4a80-bbbf-e1102a5dd220",
    "https://osmt.wgu.edu/api/collections/8f4bb4e1-46a9-4fd5-9714-aa5f0fcbc0fa",
    "https://osmt.wgu.edu/api/collections/0e594bb4-9248-4904-b375-73eb3a32e462",
    "https://osmt.wgu.edu/api/collections/9fd61243-96f0-4319-a56d-e638503e4777",
    "https://osmt.wgu.edu/api/collections/b3af69da-3480-49c5-92d7-d946497924ae",
    "https://osmt.wgu.edu/api/collections/10bb727d-ed10-40a7-b3b5-c3e1e39b128e",
    "https://osmt.wgu.edu/api/collections/aa9149fe-d86f-42af-9c3d-eadc6081fedf",
    "https://osmt.wgu.edu/api/collections/4043f96d-57fe-40da-94f5-23f90c9776f9",
    "https://osmt.wgu.edu/api/collections/85898eab-70d1-4413-93ce-8c54f57602e0",
    "https://osmt.wgu.edu/api/collections/718ab258-969e-43df-ba57-ff6fc0211267",
    "https://osmt.wgu.edu/api/collections/14b7b9db-1105-45b2-9db3-ef0fcaee4aee",
    "https://osmt.wgu.edu/api/collections/c3e0ab85-fea5-42ef-9461-cbcb9e2bc838",
    "https://osmt.wgu.edu/api/collections/e4e89fcd-e7ab-493e-9865-a0b9b071f342",
    "https://osmt.wgu.edu/api/collections/80d1b439-cdfb-4172-bb23-f480d1ed5f05",
    "https://osmt.wgu.edu/api/collections/21daebaa-3e1b-4603-b53e-8e8344e8d8d0",
    "https://osmt.wgu.edu/api/collections/6faf45a9-7b18-48bf-bb9f-4dd43e2d0e8a",
    "https://osmt.wgu.edu/api/collections/b6fe4eea-e837-490e-8d56-592e885f27be",
    "https://osmt.wgu.edu/api/collections/410b0e4b-f6df-43bc-8d3a-57c66f89dabc",
    "https://osmt.wgu.edu/api/collections/f67400c8-d00e-4bd0-bb32-2f6fe5776508",
    "https://osmt.wgu.edu/api/collections/512a4108-3778-45dc-a405-0ceab45c59c1",
    "https://osmt.wgu.edu/api/collections/a5aa189e-ba42-405e-9ba3-83e177c39c91"
 ]

### Load and Transform

In [ ]:
from neo4j import GraphDatabase
import tqdm

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def process_open_standards(tx, standard):
    # Preparing data for UNWIND
    skills_data = [{
        "skillName": skill.get("skillName", "Unknown Skill"),
        "skillStatement": skill.get("skillStatement", ""),
        "category": skill.get("category", "Unknown Category")
    } for skill in standard.get("skills", [])]

    # Merge Jurisdiction and Standard, then UNWIND skills for batch processing
    tx.run("""
    MERGE (jur:Jurisdiction {name: $author})
    MERGE (std:Standard {url: $url, title: $name, status: $status,
                         publishDate: $publishDate, creationDate: $creationDate})
    MERGE (jur)-[:HAS_STANDARD]->(std)
    WITH std
    UNWIND $skills_data AS skill
        MERGE (comp:Competency {title: skill.category})
        MERGE (skl:Skill {name: skill.skillName, description: skill.skillStatement})
        MERGE (std)-[:INCLUDES]->(comp)
        MERGE (comp)-[:HAS_SKILL]->(skl)
    """, author=standard.get("author", "Unknown Author"), url=standard.get("id", ""),
         name=standard.get("name", ""), status=standard.get("status", "Unknown Status"),
         publishDate=standard.get("publishDate", ""), creationDate=standard.get("creationDate", ""),
         skills_data=skills_data)

    def process_open_standards_optimized_with_occupations(tx, standard):
    # Preparing skills and their occupations for UNWIND
      skills_data = [{
          "skillName": skill.get("skillName", "Unknown Skill"),
          "skillStatement": skill.get("skillStatement", ""),
          "category": skill.get("category", "Unknown Category"),
          "occupations": [
              {
                  "code": occupation.get("code", "Unknown Code"),
                  "targetNodeName": occupation.get("targetNodeName", "Unknown Occupation"),
                  "frameworkName": occupation.get("frameworkName", "Unknown Framework")
              } for occupation in skill.get("occupations", [])
          ]
      } for skill in standard.get("skills", [])]

      # Merge Jurisdiction and Standard, then UNWIND skills and their occupations for batch processing
      tx.run("""
      MERGE (jur:Jurisdiction {name: $author})
      MERGE (std:Standard {url: $url, title: $name, status: $status,
                          publishDate: $publishDate, creationDate: $creationDate})
      MERGE (jur)-[:PUBLISHES]->(std)
      WITH std
      UNWIND $skills_data AS skill
          MERGE (comp:Competency {title: skill.category})
          MERGE (skl:Skill {name: skill.skillName, description: skill.skillStatement})
          MERGE (std)-[:INCLUDES]->(comp)
          MERGE (comp)-[:HAS_SKILL]->(skl)
          WITH skl, skill.occupations AS occupations
          UNWIND occupations AS occupation_data
              MERGE (occ:Occupation {code: occupation_data.code})
              ON CREATE SET occ.name = occupation_data.targetNodeName, occ.framework = occupation_data.frameworkName
              MERGE (skl)-[:RELATED_TO]->(occ)
      """, author=standard.get("author", "Unknown Author"), url=standard.get("id", ""),
          name=standard.get("name", ""), status=standard.get("status", "Unknown Status"),
          publishDate=standard.get("publishDate", ""), creationDate=standard.get("creationDate", ""),
          skills_data=skills_data)


with open("openrsd_links.txt", "r") as file:
    open_standards_links = file.read().split("\n")


tried_links = []
with driver.session() as session:
    for link in tqdm.tqdm(open_standards_links):
        response = requests.get(link, allow_redirects=True)
        if response.url:
            link = response.url

        if link in tried_links:
            continue

        link = link.split("?")[0] + "rsd.json"
        response = requests.get(link)

        tried_links.append(link)
        try:
            if response.status_code == 200:
                standard = response.json()
                session.execute_write(process_open_standards, standard)
            elif response.status_code != 404:
                print(f"Failed to fetch data for {link}, Status code: {response.status_code}")
        except Exception as e:
            print(f"Error occurred while processing {link}: {e}")

driver.close()

  0%|          | 0/34708 [00:00<?, ?it/s]


KeyboardInterrupt: 

##Curricular Analytics

###Import

In [ ]:
# Clone the GitHub repository
!git clone https://github.com/CurricularAnalytics/CA-Notebooks.git

# Use Python to find all CSV files in the cloned repository
import os

# Directory where the repository is cloned
repo_dir = '/content/CA-Notebooks'

csv_files = []
for root, dirs, files in os.walk(repo_dir):
    for file in files:
        if file.endswith('.csv'):
            csv_files.append(os.path.join(root, file))

# Print the list of CSV files
for csv_file in csv_files:
    print(csv_file)

# Example: Reading a CSV file (replace 'path_to_csv' with an actual path from the list)
import pandas as pd
df = pd.read_csv(csv_files[0], header=None)  # Assuming you want to read the first CSV file found
print(df.head(10))


fatal: destination path 'CA-Notebooks' already exists and is not an empty directory.
/content/CA-Notebooks/TransferArticulation/programs/Pima-PreEng2UA.csv
/content/CA-Notebooks/TransferArticulation/programs/UArizona-ChemE.csv
/content/CA-Notebooks/TransferArticulation/programs/Pima-UArizona-ChemE.csv
/content/CA-Notebooks/TransferArticulation/programs/Pima-PreEng.csv
/content/CA-Notebooks/TransferArticulation/programs/Pima-PreEng5UA.csv
/content/CA-Notebooks/TransferArticulation/programs/Pima-PreEng4UA.csv
/content/CA-Notebooks/TransferArticulation/original demo/transfer articulation demo/bb_data/prereq_destination.csv
/content/CA-Notebooks/TransferArticulation/original demo/transfer articulation demo/bb_data/prereq_source kctcs.csv
/content/CA-Notebooks/TransferArticulation/original demo/transfer articulation demo/bb_data/kctcs transfer plan.csv
/content/CA-Notebooks/TransferArticulation/original demo/transfer articulation demo/bb_data/equivs.csv
/content/CA-Notebooks/TransferArticul

### Load & Transform

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

def load_data_to_neo4j(df, uri, user, password):
    driver = GraphDatabase.driver(uri, auth=(user, password))

    # Find the index where course data starts, indicated by 'Course ID'
    course_header_index = df[df.iloc[:, 0] == 'Course ID'].index[0]

    # Set new headers based on the course data row
    new_headers = df.iloc[course_header_index]
    df.columns = new_headers  # Apply the new headers

    # Extract metadata using the updated DataFrame with correct headers
    metadata_keys = df.iloc[:course_header_index, 0]
    metadata_values = df.iloc[:course_header_index, 1]
    metadata = dict(zip(metadata_keys, metadata_values))

    # Now slice the DataFrame from the course data onwards
    course_df = df.iloc[course_header_index + 1:].reset_index(drop=True)

    with driver.session() as session:
        # Process metadata
        curriculum = metadata.get('Curriculum', 'Default Curriculum')
        degree_plan = metadata.get('Degree Plan', 'Default Degree Plan')
        institution = metadata['Institution']
        degree_type = metadata['Degree Type']
        system_type = metadata['System Type']
        cip = metadata['CIP']

        # Now, insert the session.run for metadata here, before iterating through course_df
        # For example:
        session.run(f"""
        MERGE (c:Curriculum {{Description: '{curriculum}', EducationLevel: '{degree_type}',
                                SystemType: '{system_type}', CIP: '{cip}'}})
        MERGE (dp:DegreePlan {{Description: '{degree_plan}'}})
        MERGE (j:Jurisdiction {{name: '{institution}'}})
        MERGE (c)-[:OFFERED_IN]->(j)
        MERGE (c)-[:INCLUDES]->(dp)
        """)

        # Then iterate over course_df for course data
        for _, row in course_df.iterrows():
            course_id = row['Course ID']
            course_name = row['Course Name']
            prefix = row['Prefix']
            number = row['Number']
            prerequisites = str(row['Prerequisites']).split(';')
            corequisites = str(row['Corequisites']).split(';')
            strict_corequisites = str(row['Strict-Corequisites']).split(';')
            credit_hours = row['Credit Hours']

            # Create or merge nodes for curriculum, degree plan, and institution
            session.run(f"""
            MERGE (course:Course {{ID: '{course_id}', Title: '{course_name}',
                                   Number: '{number}', Prefix: '{prefix}',
                                   CreditHours: '{credit_hours}'}})
            MERGE (subject:Subject {{name: '{prefix}'}})
            MERGE (subject)-[:OFFERS]->(course)
            MERGE (dp:DegreePlan {{Description: '{degree_plan}'}})
            MERGE (dp)-[:INCLUDES]->(course)
            """)

            # Create or merge the course node
            session.run(f"""
            MERGE (course:Course {{ID: '{course_id}', Title: '{course_name}',
                                   CreditHours: '{credit_hours}'}})
            MERGE (subject:Subject {{name: '{prefix}'}})
            MERGE (subject)-[:OFFERS]->(course)
            """)

            # Create relationships for prerequisites and corequisites
            for prereq in prerequisites:
                if prereq:
                    session.run(f"""
                    MATCH (course:Course {{ID: '{course_id}'}}), (prereq:Course {{ID: '{prereq}'}})
                    MERGE (course)-[:HAS_PREREQUISITE]->(prereq)
                    """)

            for coreq in corequisites:
                if coreq:
                    session.run(f"""
                    MATCH (course:Course {{ID: '{course_id}'}}), (coreq:Course {{ID: '{coreq}'}})
                    MERGE (course)-[:HAS_COREQUISITE]->(coreq)
                    """)

            for strict_coreq in strict_corequisites:
                if strict_coreq:
                    session.run(f"""
                    MATCH (course:Course {{ID: '{course_id}'}}), (strict_coreq:Course {{ID: '{strict_coreq}'}})
                    MERGE (course)-[:HAS_COREQUISITE]->(strict_coreq)
                    """)

    driver.close()

processed_files = 0  # Initialize counter

for csv_file in csv_files:
    df = pd.read_csv(csv_file, header=None, encoding='ISO-8859-1')
    processed_files += 1
    # Check if 'Course ID' is present in the DataFrame
    if 'Course ID' in df.values:
        load_data_to_neo4j(df, NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    else:
        print(f"'Course ID' not found in {csv_file}. Skipping this file.")

print(f"Processed {processed_files} files")


'Course ID' not found in /content/CA-Notebooks/TransferArticulation/original demo/transfer articulation demo/bb_data/prereq_destination.csv. Skipping this file.
'Course ID' not found in /content/CA-Notebooks/TransferArticulation/original demo/transfer articulation demo/bb_data/prereq_source kctcs.csv. Skipping this file.
'Course ID' not found in /content/CA-Notebooks/TransferArticulation/original demo/transfer articulation demo/bb_data/equivs.csv. Skipping this file.
'Course ID' not found in /content/CA-Notebooks/TransferArticulation/original demo/transfer articulation demo/bb_data/Arizona Transfer Map.csv. Skipping this file.
'Course ID' not found in /content/CA-Notebooks/TransferArticulation/original demo/transfer articulation demo/bb_data/prereq_source.csv. Skipping this file.
'Course ID' not found in /content/CA-Notebooks/TransferArticulation/original demo/transfer articulation demo/bb_data/transfer map/map_small.csv. Skipping this file.
'Course ID' not found in /content/CA-Noteboo

ClientError: {code: Neo.ClientError.Transaction.TransactionHookFailed} {message: You have exceeded the logical size limit of 400000 relationships in your database (attempt to add 1680 relationships would reach 400153 relationships). Please consider upgrading to the next tier.}

## Open Syllabus

### Import

In [ ]:
import requests
from bs4 import BeautifulSoup


# Function to scrape web page
def scrape_web_page(url, params):
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.text
    else:
        return None

# Function to parse the HTML content
def parse_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    courses_data = []
    tr_elements = soup.find_all('tr', class_='sc-8080d336-6 sc-a48d5554-0 jspDpG iOkGRx')

    for tr in tr_elements:
        course_number = tr.find('span', class_='sc-a48d5554-1 hPeoh').text
        a_element = tr.find('a', class_='sc-a48d5554-2 ehehEa')
        course_title = a_element.text
        syllabus_url = a_element['href']
        institution_date = tr.find('h3', class_='sc-a48d5554-3 hvtdQr').text.split(', ')
        institution, date = institution_date[0], institution_date[1]
        learning_objectives_elements = tr.find_all('li', class_='sc-a48d5554-7 ikbyCc')
        learning_objectives = [li.text for li in learning_objectives_elements]

        courses_data.append({
            'course_number': course_number,
            'course_title': course_title,
            'syllabus_url': syllabus_url,
            'institution': institution,
            'date': date,
            'learning_objectives': learning_objectives
        })

    return courses_data

# Main function to orchestrate scraping and parsing
def main():
    url = 'https://analytics.opensyllabus.org/record/learning-outcomes'
    params = {'size': 5}  # Example to fetch 5 records
    html_content = scrape_web_page(url, params)
    if html_content:
        html_lines = html_content.split('\n')
        for line in html_lines[:500]:
            print(line)
        courses_data = parse_html(html_content)
        # Here you can proceed with inserting data into Neo4j
        print(courses_data)  # For demonstration purposes
main()


<!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width"/><title>Open Syllabus Analytics</title><link rel="icon" type="image/png" href="/favicon.png"/><link href="https://api.mapbox.com/mapbox-gl-js/v0.51.0/mapbox-gl.css" rel="stylesheet"/><meta name="description" content="Map the college curriculum across 18.7 million syllabi"/><meta property="og:url" content="https://analytics-beta.opensyllabus.org/"/><meta property="og:type" content="website"/><meta property="og:title" content="Open Syllabus Analytics"/><meta property="og:description" content="Map the college curriculum across 18.7 million syllabi"/><meta property="og:image" content="https://analytics-beta.opensyllabus.org/open-graph-analytics.png"/><meta name="twitter:card" content="summary_large_image"/><meta property="twitter:domain" content="opensyllabus.org"/><meta property="twitter:url" content="https://analytics-beta.opensyllabus.org/"/><meta name="twitter:title" content="

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

login_email = OPEN_SYLLABUS_EMAIL


# Assuming chrome_options is already configured
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize the WebDriver
driver = webdriver.Chrome(options=chrome_options)

def disable_hotjar_and_gtm(driver):
    # Check and disable Hotjar if present
    driver.execute_script("""
    var hotjarScript = document.querySelector('script[src*="hotjar"]');
    if (hotjarScript !== null) {
        hotjarScript.remove();
    }
    """)
    # Check and disable GTM if present
    driver.execute_script("""
    var gtmScript = document.querySelector('script[src*="https://www.googletagmanager.com/gtag/js"]');
    if (gtmScript !== null) {
        gtmScript.remove();
    }
    window.dataLayer = window.dataLayer || [];
    window.dataLayer.push({
        'gtm.block': true
    });
    """)

# Navigate to the login page
#driver.get('https://analytics.opensyllabus.org/signin?redirect=/record/learning-outcomes')
driver.get('https://analytics.opensyllabus.org/verify?token=d4f534bd-22b8-4dc5-b5ae-1d31dae62417')
time.sleep(3)
driver.get('https://analytics.opensyllabus.org/record/learning-outcomes')

# Use the provided XPath to wait for the form to be present, ensuring the page has loaded
#form = WebDriverWait(driver, 10).until(
#    EC.presence_of_element_located((By.XPATH, "//*[@id='__next']/div/main/div/div/div/div[2]/div/form"))
#)

#html_content = driver.page_source

# Then find the email input within the form
#email_field = form.find_element(By.XPATH, ".//input[@type='email']")
#email_field.send_keys('d.s.porcaro.03@cantab.net')

# And the submit button within the form
#submit_button = form.find_element(By.XPATH, ".//input[@type='submit']")
#submit_button.click()

# Disable Hotjar and GTM after login
disable_hotjar_and_gtm(driver)

# Now you can navigate to the desired page and perform scraping as needed
#desired_page_url = 'https://analytics.opensyllabus.org/record/learning-outcomes?size=50'
#driver.get(desired_page_url)

WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.sc-8080d336-6.sc-a48d5554-0.jspDpG.iOkGRx"))
)

def parse_page_with_selenium(driver):
    courses_data = []
    # Locate all course rows
    tr_elements = driver.find_elements(By.CSS_SELECTOR, 'tr.sc-8080d336-6.sc-a48d5554-0.jspDpG.iOkGRx')

    for tr in tr_elements:
        course_number = tr.find_element(By.CSS_SELECTOR, 'span.sc-a48d5554-1.hPeoh').text
        a_element = tr.find_element(By.CSS_SELECTOR, 'a.sc-a48d5554-2.ehehEa')
        course_title = a_element.text
        syllabus_url = a_element.get_attribute('href')
        institution_date = tr.find_element(By.CSS_SELECTOR, 'h3.sc-a48d5554-3.hvtdQr').text.split(', ')
        institution, date = institution_date[0], institution_date[1]
        learning_objectives_elements = tr.find_elements(By.CSS_SELECTOR, 'li.sc-a48d5554-7.ikbyCc')
        learning_objectives = [li.text for li in learning_objectives_elements]

        courses_data.append({
            'course_number': course_number,
            'course_title': course_title,
            'syllabus_url': syllabus_url,
            'institution': institution,
            'date': date,
            'learning_objectives': learning_objectives
        })

    return courses_data

courses_data = parse_page_with_selenium(driver)

print(courses_data)

# Print the extracted course data
for course in courses_data:
    print(course)

driver.quit()




<html lang="en"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width"><title>Open Syllabus Analytics</title><link rel="icon" type="image/png" href="/favicon.png"><link href="https://api.mapbox.com/mapbox-gl-js/v0.51.0/mapbox-gl.css" rel="stylesheet"><meta name="description" content="Map the college curriculum across 18.7 million syllabi"><meta property="og:url" content="https://analytics-beta.opensyllabus.org/"><meta property="og:type" content="website"><meta property="og:title" content="Open Syllabus Analytics"><meta property="og:description" content="Map the college curriculum across 18.7 million syllabi"><meta property="og:image" content="https://analytics-beta.opensyllabus.org/open-graph-analytics.png"><meta name="twitter:card" content="summary_large_image"><meta property="twitter:domain" content="opensyllabus.org"><meta property="twitter:url" content="https://analytics-beta.opensyllabus.org/"><meta name="twitter:title" content="Open Syllabus Analytics"><me

###Load and Transform

In [ ]:
from py2neo import Graph, Node

# Function to insert courses and learning objectives into Neo4j
def insert_into_neo4j(courses_data):
    graph = Graph(NEO4J_URI, user=NEO4J_USER, password=NEO4J_PASSWORD)

    for course in courses_data:
        query = """
        MERGE (c:Course {id: $course_number, title: $course_title, syllabusUrl: $syllabus_url})
        MERGE (i:Institution {name: $institution, date: $date})
        MERGE (c)-[:OFFERED_BY]->(i)
        WITH c
        UNWIND $learning_objectives as lo
        MERGE (loNode:LearningObjective {description: lo})
        MERGE (loNode)-[:RELATED_TO]->(c)
        """

        graph.run(query, parameters={
            'course_number': course['course_number'],
            'course_title': course['course_title'],
            'syllabus_url': course['syllabus_url'],
            'institution': course['institution'],
            'date': course['date'],
            'learning_objectives': course['learning_objectives']
        })

# Example usage with your courses_data list
insert_into_neo4j(courses_data)

## OER Commons

###New

### Extract

### beautiful soup version (only 100 total)


In [ ]:
import requests
from bs4 import BeautifulSoup


# Fetch the main search page and extract reference URLs
def fetch_resource_urls(search_page_url):
    response = requests.get(search_page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    resource_links = soup.find_all("a", class_="item-link js-item-link")

    base_url = "https://oercommons.org"
    resource_urls = [base_url + link['href'] for link in resource_links]
    return resource_urls

# Main search page URL
#change this url based on subject, type, or search term
search_page_url = "https://oercommons.org/search?batch_size=100&sort_by=title&view_mode=summary&batch_start=200"

# Fetch resource URLs
resource_urls = fetch_resource_urls(search_page_url)
print(resource_urls[:100])
print(len(resource_urls))

['https://oercommons.org/courses/1-oa-cave-game-subtraction', 'https://oercommons.org/courses/1-oa-domino-addition', 'https://oercommons.org/courses/1-oa-doubles', 'https://oercommons.org/courses/1-oa-fact-families', 'https://oercommons.org/courses/1-oa-fact-families-with-pictures', 'https://oercommons.org/courses/1-oa-link-cube-addition', 'https://oercommons.org/courses/1-oa-measuring-blocks', 'https://oercommons.org/courses/1-oa-nbt-the-very-hungry-caterpillar', 'https://oercommons.org/courses/the-1oo-year-flood', 'https://oercommons.org/courses/1-painel-interpretativo-do-geoss%C3%ADtio-vila-pedra-pdf', 'https://oercommons.org/courses/1-ray-tracing-intuition', 'https://oercommons.org/courses/1-rotate-deformers', 'https://oercommons.org/courses/1-split-vs-average', 'https://oercommons.org/courses/1-straight-ahead-animation', 'https://oercommons.org/courses/1-string-art', 'https://oercommons.org/courses/1-theories-that-help-us-understand-families', 'https://oercommons.org/courses/1-two

### Selenium version (10000 total)

#### Get urls

Run this code for every topic type and subject to ensure you get all topics (can only get 10000 in a search).  Then combine the csvs and save as a set, then return to a list to remove duplicates.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm.notebook import tqdm

def fetch_resource_urls_with_selenium(search_page_url):
    # Chrome options
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode
    chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
    chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

    # Initialize the WebDriver with the specified options
    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 10)  # Wait object for explicit waits

    # Open the search page URL
    driver.get(search_page_url)

    # Loop to click "Load More" until it no longer appears
    while True:
        try:
            # Wait for the "Load More" button to be clickable, and then click it
            load_more_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".load-more-link.js-load-more")))
            driver.execute_script("arguments[0].click();", load_more_button)
            # Wait for a moment to allow the page content to refresh
            time.sleep(2)  # Adjust sleep time as needed
        except Exception as e:
            print("No more 'Load More' button to click or an error occurred:", e)
            break  # Exit the loop if the "Load More" button can't be found or clicked

    # Find all resource links on the page after fully loading
    resource_links = driver.find_elements(By.CSS_SELECTOR, "a.item-link.js-item-link")

    # Use tqdm to track the progress of fetching URLs
    resource_urls = []
    for link in tqdm(resource_links, total=len(resource_links), desc="Fetching URLs"):
        resource_urls.append(link.get_attribute('href'))

    # Close the browser
    driver.quit()

    return resource_urls

# Adjusted search page URL for demonstration
search_page_url = "https://oercommons.org/search?f.evaluation_type=1"

# Fetch resource URLs using Selenium
resource_urls = fetch_resource_urls_with_selenium(search_page_url)
print(f"{len(resource_urls)} URLs found")

No more 'Load More' button to click or an error occurred: Message: 
Stacktrace:
#0 0x5642f80fb7f9 <unknown>
#1 0x5642f809b3b3 <unknown>
#2 0x5642f7de3016 <unknown>
#3 0x5642f7e1781e <unknown>
#4 0x5642f7e4d8fb <unknown>
#5 0x5642f7e3aded <unknown>
#6 0x5642f7e4b9e1 <unknown>
#7 0x5642f7e3ac93 <unknown>
#8 0x5642f7e0cce4 <unknown>
#9 0x5642f7e0e4d2 <unknown>
#10 0x5642f80c7542 <unknown>
#11 0x5642f80d6ce7 <unknown>
#12 0x5642f80d69e4 <unknown>
#13 0x5642f80db13a <unknown>
#14 0x5642f80d75b9 <unknown>
#15 0x5642f80bce00 <unknown>
#16 0x5642f80ee5d2 <unknown>
#17 0x5642f80ee778 <unknown>
#18 0x5642f8106a1f <unknown>
#19 0x7e700f63fac3 <unknown>
#20 0x7e700f6d1850 <unknown>



Fetching URLs:   0%|          | 0/62 [00:00<?, ?it/s]

62 URLs found


In [ ]:
import csv
from google.colab import files

# Define the path to save the CSV file
csv_file_path = 'oer_commons_sub.csv'

# Save the URLs to a CSV file
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Optional: write a header
    writer.writerow(['Resource URLs'])
    # Write the URLs
    for url in resource_urls:
        writer.writerow([url])

# Automatically download the file to your local machine
files.download(csv_file_path)
print(f"URLs saved to {csv_file_path} and download initiated.")

#### Extract data from each url

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm
import pprint
from google.colab import files

def extract_standards(soup):
    standards = []
    # Find all divs that include the standards information
    standard_divs = soup.find_all("div", class_="user-tag item js-alignment-tag")
    for standard_div in standard_divs:
        # Extract data from meta tags
        notation = standard_div.find("meta", itemprop="targetName")['content'] if standard_div.find("meta", itemprop="targetName") else None
        title = standard_div.find("h1", class_="alignment-tag-title").get_text(strip=True) if standard_div.find("h1", class_="alignment-tag-title") else None
        domain = standard_div.find("meta", itemprop="educationalFramework")['content'] if standard_div.find("meta", itemprop="educationalFramework") else None
        learning_objective = standard_div.find("meta", itemprop="targetDescription")['content'] if standard_div.find("meta", itemprop="targetDescription") else None
        # Alignment details
        alignment = standard_div.find("span", class_="alignment-tag-info-score").get_text(strip=True) if standard_div.find("span", class_="alignment-tag-info-score") else None
        alignment_number = standard_div.find("span", class_="alignment-tag-info-evaluations-number")
        if alignment_number:
            number_text = alignment_number.get_text(strip=True).replace("users", "").replace("user", "").replace("(", "").replace(")", "")
            alignment_number = int(re.sub("[^0-9]", "", number_text)) if number_text.isdigit() else 0
        else:
            alignment_number = 0

        # Append the standardized structure to the list
        standards.append({
            "standard_notation": notation,
            "standard_title": f"{title} Grade {standard_div.find('span', class_='alignment-tag-grade').get_text(strip=True) if standard_div.find('span', class_='alignment-tag-grade') else ''}".strip(),
            "standard_domain": domain,
            "learning_objective": learning_objective,
            "standard_alignment": alignment,
            "standard_alignment_number": alignment_number
        })

    return standards

def extract_resource_data(resource_url):
    # Fetch the content of the resource page
    response = requests.get(resource_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Initialize all variables as None or with a default value to ensure they are defined
    resource_name = None
    resource_rating = None
    views = None
    resource_description = None
    subject = None
    level = None
    grade = None
    resource_type = None
    resource_author = None
    resource_date = None
    resource_license = None
    license_url = None
    language = None
    format = None

    # Proceed with extraction, checking for None at each step

    if soup.find(itemprop="name"):
        resource_name = soup.find(itemprop="name").get_text(strip=True)

    if soup.find("div", class_="stars"):
        resource_rating = soup.find("div", class_="stars")["data-rating-value"]

    views_element = soup.find("li", class_="counter-item")
    if views_element:
        views = views_element.find_all("span")[-1].get_text(strip=True)

    if soup.find(itemprop="description"):
        resource_description = soup.find(itemprop="description").get_text(strip=True)

    details = soup.find("dl", class_="materials-details-first-part")
    if details:
        if details.find(itemprop="about"):
            subject = details.find(itemprop="about").get_text(strip=True)
        if details.find("dt", string="Level:"):
            level = details.find("dt", string="Level:").find_next_sibling("dd").get_text(strip=True)
        if details.find("dt", string="Grades:"):
            grade = details.find("dt", string="Grades:").find_next_sibling("dd").get_text(strip=True)
        if details.find(itemprop="learningResourceType"):
            resource_type = details.find(itemprop="learningResourceType").get_text(strip=True)

    if details and details.find(itemprop="author"):
        resource_author = details.find(itemprop="author").get_text(strip=True)

    if details and details.find("dt", string="Date Added:"):
        resource_date = details.find("dt", string="Date Added:").find_next_sibling("dd").get_text(strip=True)

    license_detail = soup.find("div", class_="material-details-second-part")
    if license_detail and license_detail.find(rel="license"):
        license_element = license_detail.find(rel="license")
        resource_license = license_element.get_text(strip=True)
        license_url = license_element['href']

    if details and details.find(itemprop="inLanguage"):
        language = details.find(itemprop="inLanguage").get_text(strip=True)

    if details and details.find(itemprop="genre"):
        format = details.find(itemprop="genre").get_text(strip=True)

    # Extract standards using the new function
    standards = extract_standards(soup)

    # Handling Grade Level to exclude the word "grade" and just have the number
    grade_element = details.find("dt", string="Grades:")
    if grade_element and grade_element.find_next_sibling("dd"):
        grade_text = grade_element.find_next_sibling("dd").get_text(strip=True)
        grade = ''.join(filter(str.isdigit, grade_text))  # Extracts only digits

    # Compile all extracted information into a dictionary
    resource_info = {
        "Resource URL": resource_url,
        "Resource Name": resource_name,
        "Resource Rating": resource_rating,
        "Views": views,
        "Resource Description": resource_description,
        "Subject": subject,
        "Level": level,
        "Grade": grade,
        "Resource Type": resource_type,
        "Resource Author": resource_author,
        "Resource Date": resource_date,
        "Resource License": resource_license,
        "License URL": license_url,
        "Language": language,
        "Format": format,
        "Standards": standards
    }

    evaluation_data = extract_evaluation_data(soup, resource_name)
    resource_info["Evaluations"] = evaluation_data

    return resource_info

def extract_evaluation_data(soup, resource_name):
    evaluations = []
    eval_results_containers = soup.find_all("div", class_="evaluation-results js-evaluation-results")
    for eval_container in eval_results_containers:
        rubric_name_element = eval_container.find("h1", class_=["section-title", "mod-rubric", "js-rubric-header"])
        rubric_name = rubric_name_element.get_text(strip=True).split('\n')[-1] if rubric_name_element else "Unknown Rubric"
        evaluation = {
            "evaluation_name": f"OER Commons: {rubric_name} {resource_name}",  # Include Resource Name in evaluation_name
            "rubric_name": rubric_name,
            "dimensions": []
        }

        results_table = eval_container.find("table", class_="results-table")
        if results_table:
            for row in results_table.find_all("tr"):
                dimension_name = row.find("td", class_="rubric-name").get_text(strip=True)
                score_element = row.find("td", class_="alignment-scores").find("span", class_="score-value")
                score_text = score_element.get_text(strip=True) if score_element else "N/A"
                # Use regex to remove "(# users)" from the score text
                score = re.sub(r"\(\d+\susers?\)", "", score_text).strip()
                evaluation["dimensions"].append({"name": dimension_name, "score": score})

        evaluations.append(evaluation)
    return evaluations


def save_data_to_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)
    # Download file after saving
    files.download(filename)

# Assuming resource_urls is a list of URLs to process
all_resource_data = []

# Set checkpoint interval
checkpoint_interval = 1000

subset_resource_urls = resource_urls[:10]

# Extract data for each URL
progress = tqdm(subset_resource_urls, desc="Processing resource urls")
for index, url in enumerate(progress):
    try:
        resource_data = extract_resource_data(url)
        all_resource_data.append(resource_data)

        # Save and download every 1000 records
        if (index + 1) % checkpoint_interval == 0:
            checkpoint_filename = f'all_resource_data_checkpoint_{index + 1}.json'
            save_data_to_json(all_resource_data, checkpoint_filename)
            print(f"Checkpoint saved and downloaded at index {index + 1}")

        progress.set_postfix({"Last processed URL": url})

    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        # Optionally, continue to the next URL or handle the error as needed

# Save and download any remaining data at the end
#if len(all_resource_data) % checkpoint_interval != 0:
#    final_checkpoint_filename = 'all_resource_data_final_checkpoint.json'
#    save_data_to_json(all_resource_data, final_checkpoint_filename)
#    print("Final checkpoint saved and downloaded")

pprint.pprint(all_resource_data[:10])

Processing resource urls: 100%|██████████| 10/10 [00:18<00:00,  1.81s/it, Last processed URL=https://oercommons.org/courses/6-1-light-matter]


[{'Evaluations': [{'dimensions': [{'name': 'ELA', 'score': 'N/A'},
                                  {'name': 'Alignment to the Rigor of the CCSS',
                                   'score': '3'},
                                  {'name': 'Key Shifts in the CCSS',
                                   'score': '1'},
                                  {'name': 'Instructional Supports',
                                   'score': '1'},
                                  {'name': 'Assessment', 'score': '3'},
                                  {'name': 'Overall Rating for the Lesson/Unit',
                                   'score': 'E/I'},
                                  {'name': 'Alignment to the Rigor of the CCSS',
                                   'score': 'N/A'},
                                  {'name': 'Key Shifts in the CCSS',
                                   'score': 'N/A'},
                                  {'name': 'Instructional Supports',
                                   '

In [ ]:
import json

# Define the path to your JSON file
json_file_path = 'all_resource_data_final.json'

# Load the JSON data from the file
with open(json_file_path, 'r') as file:
    all_resource_data = json.load(file)

# Now, all_resource_data is a dictionary containing your data
print("Data loaded successfully.")
print("Number of records loaded:", len(all_resource_data))

### Load and Transform

In [ ]:
from neo4j import GraphDatabase
from tqdm.notebook import tqdm

# Setup Neo4j connection
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def insert_resource_evaluation_data(tx, batch_data):
    tx.run("""
    UNWIND $resources as data
    // Merge the Resource node using the unique Resource URL
    MERGE (resource:Resource {sourceURL: data.ResourceURL})
    ON CREATE SET resource += {
        name: data.ResourceName,
        rating: data.ResourceRating,
        views: data.Views,
        description: data.ResourceDescription,
        educationLevel: data.Level,
        grade: data.Grade,
        type: data.ResourceType,
        author: data.ResourceAuthor,
        date: data.ResourceDate,
        license: data.ResourceLicense,
        licenseURL: data.LicenseURL,
        language: data.Language,
        format: data.Format
    }

    // Create and link the Subject node only if the Subject is not null
    WITH data, resource
    WHERE data.Subject IS NOT NULL
    MERGE (subject:Subject {name: data.Subject})
    MERGE (resource)-[:COVERS]->(subject)

    // Handle Standards
    WITH data, resource
    UNWIND (CASE WHEN data.Standards IS NOT NULL AND size(data.Standards) > 0 THEN data.Standards ELSE [] END) AS standard
    MERGE (std:Standard {notation: standard.standard_notation})
    ON CREATE SET std += {
        title: standard.standard_title,
        domain: standard.standard_domain,
        learningObjective: standard.learning_objective,
        alignment: standard.standard_alignment,
        alignmentNumber: standard.standard_alignment_number
    }
    MERGE (resource)-[:ALIGNS_WITH]->(std)

    // Handle Evaluations if they exist and are not empty
    WITH data, resource
    UNWIND (CASE WHEN data.Evaluations IS NOT NULL AND size(data.Evaluations) > 0 THEN data.Evaluations ELSE [] END) AS evaluation
    MERGE (eval:Evaluation {name: evaluation.evaluation_name})
    ON CREATE SET eval += {
        details: evaluation.details
    }
    MERGE (resource)-[:EVALUATED_BY]->(eval)

    // Optionally, handle evaluation dimensions if part of the data
    WITH evaluation, eval
    UNWIND (CASE WHEN evaluation.dimensions IS NOT NULL AND size(evaluation.dimensions) > 0 THEN evaluation.dimensions ELSE [] END) AS dimension
    MERGE (dim:Dimension {name: dimension.name})
    MERGE (eval)-[rel:HAS_DIMENSION]->(dim)
    ON CREATE SET rel.score = dimension.score
    """, parameters={"resources": batch_data})

def process_data(all_resource_data, batch_size=100):
    with driver.session() as session:
        # Iterate over data in batches
        for i in tqdm(range(0, len(all_resource_data), batch_size)):
            batch_data = all_resource_data[i:i + batch_size]
            session.execute_write(insert_resource_evaluation_data, batch_data)
        # Closing the session within the context manager automatically closes the connection

def clean_and_format_data(resource):
    # Define default values for the resource
    default_values = {
        "Resource URL": "",
        "Resource Name": "",
        "Resource Rating": None,
        "Views": None,
        "Resource Description": "",
        "Level": "",
        "Grade": "",
        "Resource Type": "",
        "Resource Author": "",
        "Resource Date": "",
        "Resource License": "",
        "License URL": "",
        "Language": "",
        "Format": "",
        "Subject": "",
        "Standards": [],
        "Evaluations": []
    }

    # Update default values with actual values from the resource
    default_values.update(resource)

    # Clean up Evaluations to ensure they are in the correct format
    for evaluation in default_values["Evaluations"]:
        evaluation.setdefault("evaluation_name", "")
        evaluation.setdefault("rubric_name", "")
        for dimension in evaluation.get("dimensions", []):
            dimension.setdefault("name", "")
            dimension.setdefault("score", "")

    # Transform the keys to match the required format for Neo4j
    formatted_resource = {
        "ResourceURL": default_values["Resource URL"],
        "ResourceName": default_values["Resource Name"],
        "ResourceRating": default_values["Resource Rating"],
        "Views": default_values["Views"],
        "ResourceDescription": default_values["Resource Description"],
        "Level": default_values["Level"],
        "Grade": default_values["Grade"],
        "ResourceType": default_values["Resource Type"],
        "ResourceAuthor": default_values["Resource Author"],
        "ResourceDate": default_values["Resource Date"],
        "ResourceLicense": default_values["Resource License"],
        "LicenseURL": default_values["License URL"],
        "Language": default_values["Language"],
        "Format": default_values["Format"],
        "Subject": default_values["Subject"],
        "Standards": default_values["Standards"],
        "Evaluations": default_values["Evaluations"]
    }

    return formatted_resource

# Clean and format the data
formatted_data = [clean_and_format_data(resource) for resource in all_resource_data]

# Process data and insert into Neo4j
process_data(formatted_data)

  0%|          | 0/1 [00:00<?, ?it/s]

### Extract & Transform

#### Occupations

In [ ]:
import pandas as pd

# Load the CSV files
occupations_df = pd.read_csv('/content/occupations_en.csv')
research_occupations_df = pd.read_csv('/content/researchOccupationsCollection_en.csv')

# Extract the URIs from both dataframes
occupation_uris = list(occupations_df['conceptUri'].unique())
research_occupation_uris = list(research_occupations_df['conceptUri'].unique())

# Combine the lists, ensuring all entries are unique
occupation_uris = list(set(occupation_uris + research_occupation_uris))

# Now you have a list of unique URIs
print("Total Occupation URIs:", len(occupation_uris))
print("Sample Occupation URIs:", occupation_uris[:10])

Total Occupation URIs: 3039
Sample Occupation URIs: ['http://data.europa.eu/esco/occupation/9980a417-f990-495a-ae11-66a112737f3b', 'http://data.europa.eu/esco/occupation/6837611a-7093-40fb-9e7f-74947bf6f54f', 'http://data.europa.eu/esco/occupation/621b7aa1-b678-49ee-8caa-91b496f8bc5b', 'http://data.europa.eu/esco/occupation/1790c821-c582-4944-8597-d2d4a66f6a6d', 'http://data.europa.eu/esco/occupation/9358775a-3769-4327-bf25-6c0d9d69d192', 'http://data.europa.eu/esco/occupation/ceda6443-e1dd-4890-b018-42318e3abac2', 'http://data.europa.eu/esco/occupation/1801583e-34b3-4c89-97ac-db2a346b4a3e', 'http://data.europa.eu/esco/occupation/bcc2c1db-f969-4355-bebf-4878ff6c2a94', 'http://data.europa.eu/esco/occupation/76884a72-d8b8-4819-bd40-71d6420a9a42', 'http://data.europa.eu/esco/occupation/bc1c82a5-0fc3-453f-8537-9d1a6e9e85aa']


In [ ]:
import requests
from tqdm.notebook import tqdm
import json
import pprint

def fetch_occupation_details(uri):
    """Fetch detailed data for a single occupation URI."""
    url = "https://ec.europa.eu/esco/api/resource/occupation"
    headers = {"Accept": "application/json"}
    params = {
        "uri": uri,
        "language": "en"
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {'error': f'Failed to fetch {uri}', 'status_code': response.status_code}

def fetch_all_occupations(uris):
    """Fetch detailed data for a list of occupation URIs."""
    detailed_occupations = []
    for uri in tqdm(uris, desc="Fetching occupation details"):
        occupation_details = fetch_occupation_details(uri)
        detailed_occupations.append(occupation_details)
    return detailed_occupations

occupation_uris_run = occupation_uris[:5]

# Fetch detailed occupation data
detailed_occupation_data = fetch_all_occupations(occupation_uris_run)

# Save the detailed data to a JSON file
with open('/content/detailed_occupation_data.json', 'w') as f:
    json.dump(detailed_occupation_data, f, indent=4)

print("Number of Occupations", len(detailed_occupation_data))
pprint.pprint(detailed_occupation_data[:1])

Fetching occupation details:   0%|          | 0/5 [00:00<?, ?it/s]

Number of Occupations 5
[{'_embedded': {'ancestors': [{'_links': {'self': {'href': 'https://ec.europa.eu/esco/api/resource/occupation?uri=http://data.europa.eu/esco/occupation/9980a417-f990-495a-ae11-66a112737f3b&language=en',
                                                   'uri': 'http://data.europa.eu/esco/occupation/9980a417-f990-495a-ae11-66a112737f3b'}},
                               'parents': ['http://data.europa.eu/esco/isco/C8181'],
                               'title': 'glass annealer'},
                              {'_links': {'self': {'href': 'https://ec.europa.eu/esco/api/resource/occupation?uri=http://data.europa.eu/esco/isco/C8181&language=en',
                                                   'uri': 'http://data.europa.eu/esco/isco/C8181'}},
                               'parents': ['http://data.europa.eu/esco/isco/C818'],
                               'title': 'Glass and ceramics plant operators'},
                              {'_links': {'self': {'href': 'h

In [ ]:
def transform_occupations(data):
    transformed_data = []
    for item in data:
        originalID = item['uri'].split('/')[-1] if 'uri' in item else ''
        sourceURL = item.get('uri', '')
        name = item.get('preferredLabel', {}).get('en', '')
        description = item.get('description', {}).get('en', {}).get('literal', '')
        status = item.get('status', '')
        alternateName = ', '.join(item.get('alternativeLabel', {}).get('en', []))
        regulated_profession_note = item.get('_links', {}).get('regulatedProfessionNote', {}).get('title', '')
        notation = item.get('code', '')

        occupation_dict = {
            "originalID": originalID,
            "sourceURL": sourceURL,
            "name": name,
            "description": description,
            "status": status,
            "alternateName": alternateName,
            "regulatedProfessionNote": regulated_profession_note,
            "notation": notation,
            "type": "Occupation",
            "relationships": []
        }

        # Convert narrowerOccupations to structured relationships
        narrower_occupations = item.get('_links', {}).get('narrowerOccupation', [])
        for occ in narrower_occupations:
            targetID = occ['uri'].split('/')[-1]
            occupation_dict['relationships'].append({
                "type": "HAS_CHILD",
                "targetID": targetID,
            })

        transformed_data.append(occupation_dict)
    return transformed_data

# Example usage
transformed_occupations = transform_occupations(detailed_occupation_data)

print("Total number of occupations transformed:", len(transformed_occupations))
pprint.pprint((transformed_occupations)[:1])

#Optionally save output to google drive
#output_path = '/content/drive/MyDrive/ESCO/transformed_occupations.json'
#with open(output_path, 'w') as f:
#    json.dump(transformed_occupations, f)

Total number of occupations transformed: 5
[{'alternateName': 'lehr kiln operative, glass annealing kiln worker, glass '
                   'annealing oven worker, lehr operator, lehr kiln tender, '
                   'lehr operative, lehr kiln operator, lehr tender, glass '
                   'annealing kiln operator, glass annealing kiln operative, '
                   'pot annealer',
  'description': 'Glass annealers operate electric or gas kilns used to '
                 'strengthen the glass products by a heating-cooling process, '
                 'making sure the temperature is set according to '
                 'specifications. They inspect the glass products\xa0 through '
                 'the entire process to observe any flaws.',
  'name': 'glass annealer',
  'notation': '8181.7',
  'originalID': '9980a417-f990-495a-ae11-66a112737f3b',
  'regulatedProfessionNote': 'To see if and how this occupation is regulated '
                             'in EU Member States, EEA count

#### Skills

In [ ]:
import pandas as pd

# Loading all required CSV files
skills_df = pd.read_csv('/content/skills_en.csv')
broader_relations_skill_df = pd.read_csv('/content/broaderRelationsSkillPillar_en.csv')
skills_hierarchy_df = pd.read_csv('/content/skillsHierarchy_en.csv')
skill_skill_relations_df = pd.read_csv('/content/skillSkillRelations_en.csv')
research_skills_df = pd.read_csv('/content/researchSkillsCollection_en.csv')
dig_comp_skills_df = pd.read_csv('/content/digCompSkillsCollection_en.csv')
digital_skills_df = pd.read_csv('/content/digitalSkillsCollection_en.csv')
green_skills_df = pd.read_csv('/content/greenSkillsCollection_en.csv')
language_skills_df = pd.read_csv('/content/languageSkillsCollection_en.csv')
transversal_skills_df = pd.read_csv('/content/transversalSkillsCollection_en.csv')

# Extracting URIs
skills_uris = set(skills_df['conceptUri'].dropna())
broader_skills_uris = set(broader_relations_skill_df['broaderUri'].dropna())
hierarchy_skills_uris = set(skills_hierarchy_df['Level 0 URI'].dropna().unique()) | set(skills_hierarchy_df['Level 1 URI'].dropna().unique()) | set(skills_hierarchy_df['Level 2 URI'].dropna().unique()) | set(skills_hierarchy_df['Level 3 URI'].dropna().unique())
skill_skill_uris = set(skill_skill_relations_df['originalSkillUri'].dropna()) | set(skill_skill_relations_df['relatedSkillUri'].dropna())
research_skills_uris = set(research_skills_df['conceptUri'].dropna())
dig_comp_skills_uris = set(dig_comp_skills_df['conceptUri'].dropna())
digital_skills_uris = set(digital_skills_df['conceptUri'].dropna())
green_skills_uris = set(green_skills_df['conceptUri'].dropna())
language_skills_uris = set(language_skills_df['conceptUri'].dropna())
transversal_skills_uris = set(transversal_skills_df['conceptUri'].dropna())

# Combining all URIs into a single set
all_skill_uris = (skills_uris | broader_skills_uris | hierarchy_skills_uris | skill_skill_uris | research_skills_uris |
                  dig_comp_skills_uris | digital_skills_uris | green_skills_uris | language_skills_uris | transversal_skills_uris)

# Convert the set back to a list if needed
all_skill_uris_list = list(all_skill_uris)
print(f"Total unique skill URIs: {len(all_skill_uris_list)}")

In [ ]:
import requests
from tqdm.notebook import tqdm
import json
from google.colab import files

def fetch_skill_details(uri):
    """Fetch detailed data for a single skill URI."""
    url = "https://ec.europa.eu/esco/api/resource/skill"
    headers = {"Accept": "application/json"}
    params = {"uri": uri, "language": "en"}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {'error': f'Failed to fetch {uri}', 'status_code': response.status_code}

def fetch_all_skills(uris):
    """Fetch detailed data for a list of skill URIs and save periodically."""
    detailed_skills = []
    checkpoint_interval = 1000  # Save after every 1000 URIs
    for index, uri in enumerate(tqdm(uris, desc="Fetching skill details")):
        skill_details = fetch_skill_details(uri)
        detailed_skills.append(skill_details)

        # Save and download every 1000 records or at the end
        if (index + 1) % checkpoint_interval == 0 or index + 1 == len(uris):
            checkpoint_filename = f'detailed_skill_data_{index + 1}.json'
            with open(checkpoint_filename, 'w') as file:
                json.dump(detailed_skills, file, indent=4)
            files.download(checkpoint_filename)
            print(f"Checkpoint saved and downloaded at index {index + 1}")

    return detailed_skills

# List of URIs to process
skill_uris_run = all_skill_uris_list

# Fetch detailed data for each URI
detailed_skill_data = fetch_all_skills(skill_uris_run)

print("Fetched detailed data for skills:", len(detailed_skill_data))
print("Sample Detailed Skill Data:", detailed_skill_data[:2])

This takes a while.  I ran it in a couple of takes. Those because "detailed_skill_data1.json" and "detailed_skill_data2.json"

In [ ]:
import json
import pprint

# Load the first file
with open('/content/drive/MyDrive/ESCO/detailed_skill_data1.json', 'r') as file:
    detailed_skill_data1 = json.load(file)

# Load the second file
with open('/content/drive/MyDrive/ESCO/detailed_skill_data2.json', 'r') as file:
    detailed_skill_data2 = json.load(file)

# Concatenate the two lists
combined_skill_data = detailed_skill_data1 + detailed_skill_data2

# Print the length and a sample of the combined data to verify
print("Total number of skills combined:", len(combined_skill_data))
print("Sample Combined Skill Data:")
pprint.pprint((combined_skill_data)[:1])

Total number of skills combined: 15707
Sample Combined Skill Data:
[{'_embedded': {'ancestors': [{'_links': {'self': {'href': 'https://ec.europa.eu/esco/api/resource/skill?uri=http://data.europa.eu/esco/skill/1553446e-ee05-435a-91a6-acfe7f848432&language=en',
                                                   'uri': 'http://data.europa.eu/esco/skill/1553446e-ee05-435a-91a6-acfe7f848432'}},
                               'parents': ['http://data.europa.eu/esco/skill/S1.5.4',
                                           'http://data.europa.eu/esco/skill/be6ab363-3de1-427f-a8ef-85d5b0250822'],
                               'title': 'advise policy makers in healthcare'},
                              {'_links': {'self': {'href': 'https://ec.europa.eu/esco/api/resource/skill?uri=http://data.europa.eu/esco/skill/S1.5.4&language=en',
                                                   'uri': 'http://data.europa.eu/esco/skill/S1.5.4'}},
                               'parents': ['http://data.eur

In [ ]:
import pprint

def transform_skills(data):
    transformed_data = []
    for item in data:
        if 'uri' not in item:
            continue  # Skip the current iteration if 'uri' is missing

        originalID = item['uri'].split('/')[-1]
        skill_type = 'Skill'  # Default type
        skill_links = item.get('_links', {})
        skill_type_info = skill_links.get('hasSkillType', [])

        # Define type based on URI or skill type
        if 'isced-f' in item['uri']:
            skill_type = 'Subject'
        elif any(st.get('title', '').lower() == 'knowledge' for st in skill_type_info):
            skill_type = 'Knowledge'

        reuse_level = next((rl['title'] for rl in skill_links.get('hasReuseLevel', []) if 'title' in rl), '')

        skill_dict = {
            "originalID": originalID,
            "sourceURL": item['uri'],
            "name": item.get('preferredLabel', {}).get('en', ''),
            "description": item.get('description', {}).get('en', {}).get('literal', ''),
            "status": item.get('status', ''),
            "alternateName": ', '.join(item.get('alternativeLabel', {}).get('en', [])),
            "type": skill_type,
            "reuseLevel": reuse_level,
            "relationships": []
        }

        # Define relationships and required property
        relationship_mappings = {
            'narrowerSkill': ('HAS_CHILD', True),
            'hasOptionalSkill': ('HAS_CHILD', False),
            'isEssentialForSkill': ('HAS_PREREQUISITE', True),
            'isOptionalForSkill': ('HAS_PREREQUISITE', False),
            'isEssentialForOccupation': ('EMPLOYED_BY', True),
            'isOptionalForOccupation': ('EMPLOYED_BY', False)
        }

        # Process skill and occupation relationships safely checking '_links'
        for link_type, (rel_type, required) in relationship_mappings.items():
            if link_type in skill_links:
                for related in skill_links[link_type]:
                    skill_dict['relationships'].append({
                        "type": rel_type,
                        "targetID": related['uri'].split('/')[-1],
                        "required": required
                    })

        transformed_data.append(skill_dict)
    return transformed_data

# Example usage
transformed_skills = transform_skills(combined_skill_data)

print("Total number of skills transformed:", len(transformed_skills))
pprint.pprint((transformed_skills)[:1])

#Optionally save output to google drive
#output_path = '/content/drive/MyDrive/ESCO/transformed_skills.json'
#with open(output_path, 'w') as f:
#    json.dump(transformed_skills, f)


Total number of skills transformed: 14454
[{'alternateName': 'present recommendations to policy makers in healthcare, '
                   'make recommendations to health care policy makers, present '
                   'research findings to policy makers in healthcare, make '
                   'recommendations to healthcare policy makers, give '
                   'recommendations to policy makers in healthcare',
  'description': 'Present research to policy makers, health care providers, '
                 'and educators to encourage improvements in public health.',
  'name': 'advise policy makers in healthcare',
  'originalID': '1553446e-ee05-435a-91a6-acfe7f848432',
  'relationships': [{'required': True,
                     'targetID': '4a29eab1-5f02-4723-a863-e2d5a0614dfa',
                     'type': 'EMPLOYED_BY'},
                    {'required': True,
                     'targetID': '976c7f50-7f9e-4347-bbb9-f0dc842046a3',
                     'type': 'EMPLOYED_BY'},
       

#### Competencies

In [ ]:
import pandas as pd

# Load the skill groups and broader relations data
skill_groups_path = '/content/skillGroups_en.csv'
broader_relations_path = '/content/broaderRelationsSkillPillar_en.csv'
skill_groups = pd.read_csv(skill_groups_path)
broader_relations = pd.read_csv(broader_relations_path)

# Filter out entries with "isced-f" from the skill groups data
filtered_skill_groups = skill_groups[~skill_groups['conceptUri'].str.contains('isced-f')]

# Find matches where conceptUri in filtered_skill_groups matches broaderUri in broader_relations
matched_relations = broader_relations[broader_relations['broaderUri'].isin(filtered_skill_groups['conceptUri'])]

# Group by broaderUri to collect all corresponding conceptUris into a list
grouped_relations = matched_relations.groupby('broaderUri')['conceptUri'].apply(list).reset_index()

# Rename columns for clarity
grouped_relations.columns = ['conceptUri', 'targetUris']

# Merge the grouped data back to the original filtered_skill_groups
merged_competency_data = pd.merge(filtered_skill_groups, grouped_relations, on='conceptUri', how='left')

# Display the first few rows to verify
print(merged_competency_data[['conceptUri', 'preferredLabel', 'targetUris']].head())

# Optionally, save the merged data to a new CSV file for further analysis or use
#output_path = '/content/drive/MyDrive/ESCO/merged_skill_groups.csv'
#merged_competency_data.to_csv(output_path, index=False)

                                          conceptUri  \
0  http://data.europa.eu/esco/skill/00d5d1d2-c521...   
1  http://data.europa.eu/esco/skill/0101f8a4-4243...   
2  http://data.europa.eu/esco/skill/021a23e1-907e...   
3  http://data.europa.eu/esco/skill/02208098-ee91...   
4  http://data.europa.eu/esco/skill/036afa6e-cdde...   

                                      preferredLabel  \
0                  accompanying and welcoming people   
1       monitoring developments in area of expertise   
2             self-management skills and competences   
3  installing, maintaining and repairing mechanic...   
4                collaborating in teams and networks   

                                          targetUris  
0  [http://data.europa.eu/esco/skill/094e594d-83c...  
1  [http://data.europa.eu/esco/skill/e2a5e046-335...  
2  [http://data.europa.eu/esco/skill/14c41899-022...  
3  [http://data.europa.eu/esco/skill/03efa398-a1c...  
4  [http://data.europa.eu/esco/skill/60c78287-22e..

In [ ]:
import pandas as pd

def transform_competencies(data):
    competencies = []
    for _, row in data.iterrows():
        # Skip processing if conceptUri contains 'isced-f'
        if 'isced-f' in row['conceptUri']:
            continue

        # Initialize the dictionary for the current competency
        competency_dict = {
            "originalID": row['conceptUri'].split('/')[-1],
            "sourceURL": row['conceptUri'],
            "name": row['preferredLabel'],
            "description": row.get('description', ''),
            "status": row['status'],
            "code": row.get('code', ''),
            "type": "Competency",
            "alternateNames": ', '.join(row['altLabels'].split('|')) if pd.notna(row['altLabels']) else '',
            "relationships": []
        }

        # Process targetUris if they exist and are not NaN
        target_uris = row.get('targetUris')
        if isinstance(target_uris, list):
            for uri in target_uris:
                competency_dict['relationships'].append({
                    "type": "HAS_CHILD",
                    "targetID": uri.split('/')[-1]
                })

        competencies.append(competency_dict)

    return competencies

# Example usage
transformed_competencies = transform_competencies(merged_competency_data)

print("Total number of competencies transformed:", len(transformed_competencies))
pprint.pprint(transformed_competencies[0])

# Optionally save output to google drive
# output_path = '/content/drive/MyDrive/ESCO/transformed_competencies.json'
#with open(output_path, 'w') as f:
#    json.dump(transformed_competencies, f)

Total number of competencies transformed: 420
{'alternateNames': '',
 'code': 'S3.4.2',
 'description': 'Welcoming, accompanying or supporting clients, patrons, '
                'visitors and others by helping them to feel at ease, and '
                'assisting them with activities such as finding seats, making '
                'reservations and using facilities at social events, outings, '
                'or trips.',
 'name': 'accompanying and welcoming people',
 'originalID': '00d5d1d2-c521-4c58-9df5-2939c0ce9adb',
 'relationships': [{'targetID': '094e594d-83cc-4480-bad8-fae0ec2ed8ab',
                    'type': 'HAS_CHILD'},
                   {'targetID': '13fe62b4-ea25-46e1-91f8-4201161c0107',
                    'type': 'HAS_CHILD'},
                   {'targetID': '1b45b91c-0949-45cf-a6d8-040a62423060',
                    'type': 'HAS_CHILD'},
                   {'targetID': '1ce3af48-b15f-4851-8db6-cbbdc49ad19b',
                    'type': 'HAS_CHILD'},
               

### Load

In [ ]:

# Load data from JSON files
with open('/content/drive/MyDrive/ESCO/transformed_occupations.json', 'r') as f:
    all_occupations = json.load(f)
with open('/content/drive/MyDrive/ESCO/transformed_skills.json', 'r') as f:
    all_skills = json.load(f)
with open('/content/drive/MyDrive/ESCO/transformed_competencies.json', 'r') as f:
    all_competencies = json.load(f)

print("Number of Occupations:", len(all_occupations))
print("Number of Skills:", len(all_skills))
print("Number of Competencies:", len(all_competencies))



Number of Occupations: 2933
Number of Skills: 14454
Number of Competencies: 420


In [ ]:
from neo4j import GraphDatabase
from tqdm.notebook import tqdm

# Setup Neo4j connection
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def create_nodes(tx, data):
    create_query = """
    UNWIND $data as item
    CALL apoc.merge.node([item.type], {originalID: item.originalID}, item.props, item.props)
    YIELD node
    RETURN count(node)
    """
    tx.run(create_query, parameters={'data': data})

def create_relationships(tx, data):
    relationship_query = """
    UNWIND $data as item
    MATCH (entity {originalID: item.originalID})
    UNWIND item.relationships as rel
    MATCH (target {originalID: rel.targetID})
    CALL apoc.create.relationship(entity, rel.type, rel.props, target)
    YIELD rel as createdRel
    RETURN count(createdRel)
    """
    tx.run(relationship_query, parameters={'data': data})

def load_data(session, all_data, batch_size=100):
    total = len(all_data)
    # First, create all nodes
    node_data = [{
        'type': item['type'],
        'originalID': item['originalID'],
        'props': {k: v for k, v in item.items() if v is not None and v != "" and k not in ['relationships', 'type', 'originalID']}
    } for item in all_data if 'originalID' in item]

    for i in tqdm(range(0, total, batch_size)):
        batch_data = node_data[i:i+batch_size]
        session.execute_write(create_nodes, batch_data)

    # Then, create all relationships
    relationship_data = [{
        'originalID': item['originalID'],
        'relationships': [{
            'targetID': rel['targetID'],
            'type': rel['type'],
            'props': {'required': rel.get('required')} if 'required' in rel else {}
        } for rel in item.get('relationships', []) if 'targetID' in rel]
    } for item in all_data if 'originalID' in item and item.get('relationships')]

    for i in tqdm(range(0, total, batch_size)):
        batch_data = relationship_data[i:i+batch_size]
        session.execute_write(create_relationships, batch_data)

# Usage example with error handling
sample_all_occupations = all_occupations #[:100]
sample_all_skills = all_skills #[:100]
sample_all_competencies = all_competencies #[:100]

# Usage example with error handling
with driver.session() as session:
    try:
        combined_data = sample_all_occupations + sample_all_skills + sample_all_competencies
        load_data(session, combined_data)
    except Exception as e:
        print(f"An error occurred: {e}")

driver.close()

  0%|          | 0/179 [00:00<?, ?it/s]

## ESCO

### Extract & Transform

#### Occupations

In [ ]:
import pandas as pd

# Load the CSV files
occupations_df = pd.read_csv('/content/occupations_en.csv')
research_occupations_df = pd.read_csv('/content/researchOccupationsCollection_en.csv')

# Extract the URIs from both dataframes
occupation_uris = list(occupations_df['conceptUri'].unique())
research_occupation_uris = list(research_occupations_df['conceptUri'].unique())

# Combine the lists, ensuring all entries are unique
occupation_uris = list(set(occupation_uris + research_occupation_uris))

# Now you have a list of unique URIs
print("Total Occupation URIs:", len(occupation_uris))
print("Sample Occupation URIs:", occupation_uris[:10])

Total Occupation URIs: 3039
Sample Occupation URIs: ['http://data.europa.eu/esco/occupation/9980a417-f990-495a-ae11-66a112737f3b', 'http://data.europa.eu/esco/occupation/6837611a-7093-40fb-9e7f-74947bf6f54f', 'http://data.europa.eu/esco/occupation/621b7aa1-b678-49ee-8caa-91b496f8bc5b', 'http://data.europa.eu/esco/occupation/1790c821-c582-4944-8597-d2d4a66f6a6d', 'http://data.europa.eu/esco/occupation/9358775a-3769-4327-bf25-6c0d9d69d192', 'http://data.europa.eu/esco/occupation/ceda6443-e1dd-4890-b018-42318e3abac2', 'http://data.europa.eu/esco/occupation/1801583e-34b3-4c89-97ac-db2a346b4a3e', 'http://data.europa.eu/esco/occupation/bcc2c1db-f969-4355-bebf-4878ff6c2a94', 'http://data.europa.eu/esco/occupation/76884a72-d8b8-4819-bd40-71d6420a9a42', 'http://data.europa.eu/esco/occupation/bc1c82a5-0fc3-453f-8537-9d1a6e9e85aa']


In [ ]:
import requests
from tqdm.notebook import tqdm
import json
import pprint

def fetch_occupation_details(uri):
    """Fetch detailed data for a single occupation URI."""
    url = "https://ec.europa.eu/esco/api/resource/occupation"
    headers = {"Accept": "application/json"}
    params = {
        "uri": uri,
        "language": "en"
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {'error': f'Failed to fetch {uri}', 'status_code': response.status_code}

def fetch_all_occupations(uris):
    """Fetch detailed data for a list of occupation URIs."""
    detailed_occupations = []
    for uri in tqdm(uris, desc="Fetching occupation details"):
        occupation_details = fetch_occupation_details(uri)
        detailed_occupations.append(occupation_details)
    return detailed_occupations

occupation_uris_run = occupation_uris #[:5]

# Fetch detailed occupation data
detailed_occupation_data = fetch_all_occupations(occupation_uris_run)

# Save the detailed data to a JSON file
with open('/content/detailed_occupation_data.json', 'w') as f:
    json.dump(detailed_occupation_data, f, indent=4)

print("Number of Occupations", len(detailed_occupation_data))
pprint.pprint(detailed_occupation_data[:1])

Fetching occupation details:   0%|          | 0/5 [00:00<?, ?it/s]

Number of Occupations 5
[{'_embedded': {'ancestors': [{'_links': {'self': {'href': 'https://ec.europa.eu/esco/api/resource/occupation?uri=http://data.europa.eu/esco/occupation/9980a417-f990-495a-ae11-66a112737f3b&language=en',
                                                   'uri': 'http://data.europa.eu/esco/occupation/9980a417-f990-495a-ae11-66a112737f3b'}},
                               'parents': ['http://data.europa.eu/esco/isco/C8181'],
                               'title': 'glass annealer'},
                              {'_links': {'self': {'href': 'https://ec.europa.eu/esco/api/resource/occupation?uri=http://data.europa.eu/esco/isco/C8181&language=en',
                                                   'uri': 'http://data.europa.eu/esco/isco/C8181'}},
                               'parents': ['http://data.europa.eu/esco/isco/C818'],
                               'title': 'Glass and ceramics plant operators'},
                              {'_links': {'self': {'href': 'h

In [ ]:
def transform_occupations(data):
    transformed_data = []
    for item in data:
        originalID = item['uri'].split('/')[-1] if 'uri' in item else ''
        sourceURL = item.get('uri', '')
        name = item.get('preferredLabel', {}).get('en', '')
        description = item.get('description', {}).get('en', {}).get('literal', '')
        status = item.get('status', '')
        alternateName = ', '.join(item.get('alternativeLabel', {}).get('en', []))
        regulated_profession_note = item.get('_links', {}).get('regulatedProfessionNote', {}).get('title', '')
        notation = item.get('code', '')

        occupation_dict = {
            "originalID": originalID,
            "sourceURL": sourceURL,
            "name": name,
            "description": description,
            "status": status,
            "alternateName": alternateName,
            "regulatedProfessionNote": regulated_profession_note,
            "notation": notation,
            "type": "Occupation",
            "relationships": []
        }

        # Convert narrowerOccupations to structured relationships
        narrower_occupations = item.get('_links', {}).get('narrowerOccupation', [])
        for occ in narrower_occupations:
            targetID = occ['uri'].split('/')[-1]
            occupation_dict['relationships'].append({
                "type": "HAS_CHILD",
                "targetID": targetID,
            })

        transformed_data.append(occupation_dict)
    return transformed_data

# Example usage
transformed_occupations = transform_occupations(detailed_occupation_data)

print("Total number of occupations transformed:", len(transformed_occupations))
pprint.pprint((transformed_occupations)[:1])

#Optionally save output to google drive
#output_path = '/content/drive/MyDrive/ESCO/transformed_occupations.json'
#with open(output_path, 'w') as f:
#    json.dump(transformed_occupations, f)

Total number of occupations transformed: 5
[{'alternateName': 'lehr kiln operative, glass annealing kiln worker, glass '
                   'annealing oven worker, lehr operator, lehr kiln tender, '
                   'lehr operative, lehr kiln operator, lehr tender, glass '
                   'annealing kiln operator, glass annealing kiln operative, '
                   'pot annealer',
  'description': 'Glass annealers operate electric or gas kilns used to '
                 'strengthen the glass products by a heating-cooling process, '
                 'making sure the temperature is set according to '
                 'specifications. They inspect the glass products\xa0 through '
                 'the entire process to observe any flaws.',
  'name': 'glass annealer',
  'notation': '8181.7',
  'originalID': '9980a417-f990-495a-ae11-66a112737f3b',
  'regulatedProfessionNote': 'To see if and how this occupation is regulated '
                             'in EU Member States, EEA count

#### Skills

In [ ]:
import pandas as pd

# Loading all required CSV files
skills_df = pd.read_csv('/content/skills_en.csv')
broader_relations_skill_df = pd.read_csv('/content/broaderRelationsSkillPillar_en.csv')
skills_hierarchy_df = pd.read_csv('/content/skillsHierarchy_en.csv')
skill_skill_relations_df = pd.read_csv('/content/skillSkillRelations_en.csv')
research_skills_df = pd.read_csv('/content/researchSkillsCollection_en.csv')
dig_comp_skills_df = pd.read_csv('/content/digCompSkillsCollection_en.csv')
digital_skills_df = pd.read_csv('/content/digitalSkillsCollection_en.csv')
green_skills_df = pd.read_csv('/content/greenSkillsCollection_en.csv')
language_skills_df = pd.read_csv('/content/languageSkillsCollection_en.csv')
transversal_skills_df = pd.read_csv('/content/transversalSkillsCollection_en.csv')

# Extracting URIs
skills_uris = set(skills_df['conceptUri'].dropna())
broader_skills_uris = set(broader_relations_skill_df['broaderUri'].dropna())
hierarchy_skills_uris = set(skills_hierarchy_df['Level 0 URI'].dropna().unique()) | set(skills_hierarchy_df['Level 1 URI'].dropna().unique()) | set(skills_hierarchy_df['Level 2 URI'].dropna().unique()) | set(skills_hierarchy_df['Level 3 URI'].dropna().unique())
skill_skill_uris = set(skill_skill_relations_df['originalSkillUri'].dropna()) | set(skill_skill_relations_df['relatedSkillUri'].dropna())
research_skills_uris = set(research_skills_df['conceptUri'].dropna())
dig_comp_skills_uris = set(dig_comp_skills_df['conceptUri'].dropna())
digital_skills_uris = set(digital_skills_df['conceptUri'].dropna())
green_skills_uris = set(green_skills_df['conceptUri'].dropna())
language_skills_uris = set(language_skills_df['conceptUri'].dropna())
transversal_skills_uris = set(transversal_skills_df['conceptUri'].dropna())

# Combining all URIs into a single set
all_skill_uris = (skills_uris | broader_skills_uris | hierarchy_skills_uris | skill_skill_uris | research_skills_uris |
                  dig_comp_skills_uris | digital_skills_uris | green_skills_uris | language_skills_uris | transversal_skills_uris)

# Convert the set back to a list if needed
all_skill_uris_list = list(all_skill_uris)
print(f"Total unique skill URIs: {len(all_skill_uris_list)}")

In [ ]:
import requests
from tqdm.notebook import tqdm
import json
from google.colab import files

def fetch_skill_details(uri):
    """Fetch detailed data for a single skill URI."""
    url = "https://ec.europa.eu/esco/api/resource/skill"
    headers = {"Accept": "application/json"}
    params = {"uri": uri, "language": "en"}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {'error': f'Failed to fetch {uri}', 'status_code': response.status_code}

def fetch_all_skills(uris):
    """Fetch detailed data for a list of skill URIs and save periodically."""
    detailed_skills = []
    checkpoint_interval = 1000  # Save after every 1000 URIs
    for index, uri in enumerate(tqdm(uris, desc="Fetching skill details")):
        skill_details = fetch_skill_details(uri)
        detailed_skills.append(skill_details)

        # Save and download every 1000 records or at the end
        if (index + 1) % checkpoint_interval == 0 or index + 1 == len(uris):
            checkpoint_filename = f'detailed_skill_data_{index + 1}.json'
            with open(checkpoint_filename, 'w') as file:
                json.dump(detailed_skills, file, indent=4)
            files.download(checkpoint_filename)
            print(f"Checkpoint saved and downloaded at index {index + 1}")

    return detailed_skills

# List of URIs to process
skill_uris_run = all_skill_uris_list

# Fetch detailed data for each URI
detailed_skill_data = fetch_all_skills(skill_uris_run)

print("Fetched detailed data for skills:", len(detailed_skill_data))
print("Sample Detailed Skill Data:", detailed_skill_data[:2])

This takes a while.  I ran it in a couple of takes. Those became "detailed_skill_data1.json" and "detailed_skill_data2.json"

In [ ]:
import json
import pprint

# Load the first file
with open('/content/drive/MyDrive/ESCO/detailed_skill_data1.json', 'r') as file:
    detailed_skill_data1 = json.load(file)

# Load the second file
with open('/content/drive/MyDrive/ESCO/detailed_skill_data2.json', 'r') as file:
    detailed_skill_data2 = json.load(file)

# Concatenate the two lists
combined_skill_data = detailed_skill_data1 + detailed_skill_data2

# Print the length and a sample of the combined data to verify
print("Total number of skills combined:", len(combined_skill_data))
print("Sample Combined Skill Data:")
pprint.pprint((combined_skill_data)[:1])

Total number of skills combined: 15707
Sample Combined Skill Data:
[{'_embedded': {'ancestors': [{'_links': {'self': {'href': 'https://ec.europa.eu/esco/api/resource/skill?uri=http://data.europa.eu/esco/skill/1553446e-ee05-435a-91a6-acfe7f848432&language=en',
                                                   'uri': 'http://data.europa.eu/esco/skill/1553446e-ee05-435a-91a6-acfe7f848432'}},
                               'parents': ['http://data.europa.eu/esco/skill/S1.5.4',
                                           'http://data.europa.eu/esco/skill/be6ab363-3de1-427f-a8ef-85d5b0250822'],
                               'title': 'advise policy makers in healthcare'},
                              {'_links': {'self': {'href': 'https://ec.europa.eu/esco/api/resource/skill?uri=http://data.europa.eu/esco/skill/S1.5.4&language=en',
                                                   'uri': 'http://data.europa.eu/esco/skill/S1.5.4'}},
                               'parents': ['http://data.eur

In [ ]:
import pprint

def transform_skills(data):
    transformed_data = []
    for item in data:
        if 'uri' not in item:
            continue  # Skip the current iteration if 'uri' is missing

        originalID = item['uri'].split('/')[-1]
        skill_type = 'Skill'  # Default type
        skill_links = item.get('_links', {})
        skill_type_info = skill_links.get('hasSkillType', [])

        # Define type based on URI or skill type
        if 'isced-f' in item['uri']:
            skill_type = 'Subject'
        elif any(st.get('title', '').lower() == 'knowledge' for st in skill_type_info):
            skill_type = 'Knowledge'

        reuse_level = next((rl['title'] for rl in skill_links.get('hasReuseLevel', []) if 'title' in rl), '')

        skill_dict = {
            "originalID": originalID,
            "sourceURL": item['uri'],
            "name": item.get('preferredLabel', {}).get('en', ''),
            "description": item.get('description', {}).get('en', {}).get('literal', ''),
            "status": item.get('status', ''),
            "alternateName": ', '.join(item.get('alternativeLabel', {}).get('en', [])),
            "type": skill_type,
            "reuseLevel": reuse_level,
            "relationships": []
        }

        # Define relationships and required property
        relationship_mappings = {
            'narrowerSkill': ('HAS_CHILD', True),
            'hasOptionalSkill': ('HAS_CHILD', False),
            'isEssentialForSkill': ('HAS_PREREQUISITE', True),
            'isOptionalForSkill': ('HAS_PREREQUISITE', False),
            'isEssentialForOccupation': ('EMPLOYED_BY', True),
            'isOptionalForOccupation': ('EMPLOYED_BY', False)
        }

        # Process skill and occupation relationships safely checking '_links'
        for link_type, (rel_type, required) in relationship_mappings.items():
            if link_type in skill_links:
                for related in skill_links[link_type]:
                    target_id = related['uri'].split('/')[-1]
                    relationship = {
                        "type": rel_type,
                        "targetID": target_id,
                        "required": required
                    }

                    # Check if relationship is HAS_PREREQUISITE and swap direction
                    if rel_type == 'HAS_PREREQUISITE':
                        # Swap the direction for HAS_PREREQUISITE relationships
                        relationship["sourceID"] = skill_dict['originalID']
                        relationship["targetID"] = target_id
                        # Check if the reversed HAS_PREREQUISITE already exists
                        if not any(rel for rel in skill_dict['relationships'] if rel['type'] == 'HAS_PREREQUISITE' and rel['targetID'] == skill_dict['originalID'] and rel['sourceID'] == target_id):
                            skill_dict['relationships'].append(relationship)
                    else:
                        # Check if a HAS_PREREQUISITE relationship already exists
                        if not any(rel for rel in skill_dict['relationships'] if rel['type'] == 'HAS_PREREQUISITE' and rel['targetID'] == target_id):
                            skill_dict['relationships'].append(relationship)


        transformed_data.append(skill_dict)
    return transformed_data

# Example usage
transformed_skills = transform_skills(combined_skill_data)

print("Total number of skills transformed:", len(transformed_skills))
pprint.pprint((transformed_skills)[:1])

#Optionally save output to google drive
#output_path = '/content/drive/MyDrive/ESCO/transformed_skills.json'
#with open(output_path, 'w') as f:
#    json.dump(transformed_skills, f)


Total number of skills transformed: 14454
[{'alternateName': 'present recommendations to policy makers in healthcare, '
                   'make recommendations to health care policy makers, present '
                   'research findings to policy makers in healthcare, make '
                   'recommendations to healthcare policy makers, give '
                   'recommendations to policy makers in healthcare',
  'description': 'Present research to policy makers, health care providers, '
                 'and educators to encourage improvements in public health.',
  'name': 'advise policy makers in healthcare',
  'originalID': '1553446e-ee05-435a-91a6-acfe7f848432',
  'relationships': [{'required': True,
                     'targetID': '4a29eab1-5f02-4723-a863-e2d5a0614dfa',
                     'type': 'EMPLOYED_BY'},
                    {'required': True,
                     'targetID': '976c7f50-7f9e-4347-bbb9-f0dc842046a3',
                     'type': 'EMPLOYED_BY'},
       

#### Competencies

In [ ]:
import pandas as pd

# Load the skill groups and broader relations data
skill_groups_path = '/content/skillGroups_en.csv'
broader_relations_path = '/content/broaderRelationsSkillPillar_en.csv'
skill_groups = pd.read_csv(skill_groups_path)
broader_relations = pd.read_csv(broader_relations_path)

# Filter out entries with "isced-f" from the skill groups data
filtered_skill_groups = skill_groups[~skill_groups['conceptUri'].str.contains('isced-f')]

# Find matches where conceptUri in filtered_skill_groups matches broaderUri in broader_relations
matched_relations = broader_relations[broader_relations['broaderUri'].isin(filtered_skill_groups['conceptUri'])]

# Group by broaderUri to collect all corresponding conceptUris into a list
grouped_relations = matched_relations.groupby('broaderUri')['conceptUri'].apply(list).reset_index()

# Rename columns for clarity
grouped_relations.columns = ['conceptUri', 'targetUris']

# Merge the grouped data back to the original filtered_skill_groups
merged_competency_data = pd.merge(filtered_skill_groups, grouped_relations, on='conceptUri', how='left')

# Display the first few rows to verify
print(merged_competency_data[['conceptUri', 'preferredLabel', 'targetUris']].head())

# Optionally, save the merged data to a new CSV file for further analysis or use
#output_path = '/content/drive/MyDrive/ESCO/merged_skill_groups.csv'
#merged_competency_data.to_csv(output_path, index=False)

                                          conceptUri  \
0  http://data.europa.eu/esco/skill/00d5d1d2-c521...   
1  http://data.europa.eu/esco/skill/0101f8a4-4243...   
2  http://data.europa.eu/esco/skill/021a23e1-907e...   
3  http://data.europa.eu/esco/skill/02208098-ee91...   
4  http://data.europa.eu/esco/skill/036afa6e-cdde...   

                                      preferredLabel  \
0                  accompanying and welcoming people   
1       monitoring developments in area of expertise   
2             self-management skills and competences   
3  installing, maintaining and repairing mechanic...   
4                collaborating in teams and networks   

                                          targetUris  
0  [http://data.europa.eu/esco/skill/094e594d-83c...  
1  [http://data.europa.eu/esco/skill/e2a5e046-335...  
2  [http://data.europa.eu/esco/skill/14c41899-022...  
3  [http://data.europa.eu/esco/skill/03efa398-a1c...  
4  [http://data.europa.eu/esco/skill/60c78287-22e..

In [ ]:
import pandas as pd

def transform_competencies(data):
    competencies = []
    for _, row in data.iterrows():
        # Skip processing if conceptUri contains 'isced-f'
        if 'isced-f' in row['conceptUri']:
            continue

        # Initialize the dictionary for the current competency
        competency_dict = {
            "originalID": row['conceptUri'].split('/')[-1],
            "sourceURL": row['conceptUri'],
            "name": row['preferredLabel'],
            "description": row.get('description', ''),
            "status": row['status'],
            "code": row.get('code', ''),
            "type": "Competency",
            "alternateNames": ', '.join(row['altLabels'].split('|')) if pd.notna(row['altLabels']) else '',
            "relationships": []
        }

        # Process targetUris if they exist and are not NaN
        target_uris = row.get('targetUris')
        if isinstance(target_uris, list):
            for uri in target_uris:
                competency_dict['relationships'].append({
                    "type": "HAS_CHILD",
                    "targetID": uri.split('/')[-1]
                })

        competencies.append(competency_dict)

    return competencies

# Example usage
transformed_competencies = transform_competencies(merged_competency_data)

print("Total number of competencies transformed:", len(transformed_competencies))
pprint.pprint(transformed_competencies[0])

# Optionally save output to google drive
# output_path = '/content/drive/MyDrive/ESCO/transformed_competencies.json'
#with open(output_path, 'w') as f:
#    json.dump(transformed_competencies, f)

Total number of competencies transformed: 420
{'alternateNames': '',
 'code': 'S3.4.2',
 'description': 'Welcoming, accompanying or supporting clients, patrons, '
                'visitors and others by helping them to feel at ease, and '
                'assisting them with activities such as finding seats, making '
                'reservations and using facilities at social events, outings, '
                'or trips.',
 'name': 'accompanying and welcoming people',
 'originalID': '00d5d1d2-c521-4c58-9df5-2939c0ce9adb',
 'relationships': [{'targetID': '094e594d-83cc-4480-bad8-fae0ec2ed8ab',
                    'type': 'HAS_CHILD'},
                   {'targetID': '13fe62b4-ea25-46e1-91f8-4201161c0107',
                    'type': 'HAS_CHILD'},
                   {'targetID': '1b45b91c-0949-45cf-a6d8-040a62423060',
                    'type': 'HAS_CHILD'},
                   {'targetID': '1ce3af48-b15f-4851-8db6-cbbdc49ad19b',
                    'type': 'HAS_CHILD'},
               

### Load

In [ ]:
import json

# Load data from JSON files
with open('/content/drive/MyDrive/ESCO/transformed_occupations.json', 'r') as f:
    all_occupations = json.load(f)
with open('/content/drive/MyDrive/ESCO/transformed_skills.json', 'r') as f:
    all_skills = json.load(f)
with open('/content/drive/MyDrive/ESCO/transformed_competencies.json', 'r') as f:
    all_competencies = json.load(f)

print("Number of Occupations:", len(all_occupations))
print("Number of Skills:", len(all_skills))
print("Number of Competencies:", len(all_competencies))



Number of Occupations: 2933
Number of Skills: 14454
Number of Competencies: 420


The loading to Neo4J takes a REALLY long time in production (a few days), even though it's fairly quick in dev.  I tried to optimize where possible, but this is the best it can do.

In [ ]:
from neo4j import GraphDatabase
from tqdm.notebook import tqdm

# Setup Neo4j connection
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def create_nodes(tx, data):
    create_query = """
    UNWIND $data as item
    CALL apoc.merge.node([item.type], {originalID: item.originalID}, item.props, item.props)
    YIELD node
    RETURN count(node)
    """
    tx.run(create_query, parameters={'data': data})

def create_relationships(tx, data, rel_type):
    relationship_query = f"""
    UNWIND $data as item
    MATCH (entity {{originalID: item.originalID}})
    WITH entity, item
    UNWIND item.relationships as rel
    MATCH (target {{originalID: rel.targetID}})
    MERGE (entity)-[r:{rel_type}]->(target)
    ON CREATE SET r += rel.props
    RETURN count(r)
    """
    tx.run(relationship_query, parameters={'data': data})

def load_data(session, all_data, batch_size=100, initial_progress=None):
    total = len(all_data)

    if initial_progress is None:
        initial_progress = {}

    # First, create all nodes
    node_data = [{
        'type': item['type'],
        'originalID': item['originalID'],
        'props': {k: v for k, v in item.items() if v is not None and v != "" and k not in ['relationships', 'type', 'originalID']}
    } for item in all_data if 'originalID' in item]

    if 'nodes' not in initial_progress or initial_progress['nodes'] == 'completed':
        for i in tqdm(range(0, total, batch_size), desc="Creating Nodes"):
            batch_data = node_data[i:i+batch_size]
            session.execute_write(create_nodes, batch_data)
        initial_progress['nodes'] = 'completed'

    # Separate relationships by type
    rel_types = ["HAS_PREREQUISITE", "HAS_CHILD", "EMPLOYED_BY"]
    relationship_data = {rel_type: [] for rel_type in rel_types}

    for item in all_data:
        if 'relationships' in item:
            for rel in item['relationships']:
                if rel['type'] in rel_types:
                    relationship_data[rel['type']].append({
                        'originalID': item['originalID'],
                        'relationships': [{
                            'targetID': rel['targetID'],
                            'props': {'required': rel.get('required')} if 'required' in rel else {}
                        }]
                    })

    # Create relationships in batches by type
    for rel_type, data in relationship_data.items():
        if rel_type not in initial_progress:
            initial_progress[rel_type] = 0
        for i in tqdm(range(initial_progress[rel_type], len(data), batch_size), desc=f"Creating Relationships - {rel_type}"):
            batch_data = data[i:i+batch_size]
            try:
                session.execute_write(create_relationships, batch_data, rel_type)
                initial_progress[rel_type] = i + batch_size
            except Exception as e:
                print(f"An error occurred at batch {i}: {e}")
                raise

# Usage example with error handling
sample_all_occupations = all_occupations
sample_all_skills = all_skills
sample_all_competencies = all_competencies

initial_progress = {
    'nodes': 0,
    'HAS_PREREQUISITE': 0, # starting point for HAS_PREREQUISITE relationships
    'HAS_CHILD': 0,
    'EMPLOYED_BY': 0
}

with driver.session() as session:
    try:
        combined_data = sample_all_occupations + sample_all_skills + sample_all_competencies
        load_data(session, combined_data, initial_progress=initial_progress)
    except Exception as e:
        print(f"An error occurred: {e}")

driver.close()

Creating Relationships - HAS_PREREQUISITE:   0%|          | 0/64 [00:00<?, ?it/s]

Creating Relationships - HAS_CHILD:   0%|          | 0/288 [00:00<?, ?it/s]

KeyboardInterrupt: 

## WWC

### Extract and Transform

In [ ]:
import re
import pandas as pd
import json
from pprint import pprint
from tqdm.notebook import tqdm

dtype_spec = {
    'i_Intervention_Page_URL': 'str',
    'f_Outcome_MeasureID': 'str',
    's_StudyID': 'str',
    'i_InterventionID': 'str',
    's_ProductID': 'str',
    224: 'str',
    229: 'str',
    230: 'str',
    252: 'str',
    280: 'boolean',
    284: 'str'
}

# Load the CSV file to a DataFrame
df_wwc = pd.read_csv('/content/Interventions_Studies_And_Findings.csv', dtype=dtype_spec)


def map_effectiveness(effectiveness):
    mapping = {
        "Negative Effects": -2,
        "Potentially Negative Effects": -1,
        "No Discernible Effects": 0,
        "Mixed Effects": 1,
        "Positive Effects": 2,
        "Potentially Positive Effects": 3,
        "Unknown": "Unknown"
    }
    return mapping.get(effectiveness, "Unknown")

def map_evidence_quality(evidenceQuality):
    mapping = {
        "Meets WWC standards without reservations": 2,
        "Meets WWC standards with reservations": 1,
        "Does not meet WWC standards": 0,
        "Unknown": "Unknown"
    }
    return mapping.get(evidenceQuality, "Unknown")

def format_percentage(value):
    """Format numeric values or ranges as percentages."""
    if isinstance(value, str) and '..' in value:
        # Handling range values, e.g., "11..74" to mean "11% to 74%"
        return value.replace('..', '% to ') + '%'
    elif pd.notnull(value):
        # Handling single percentage values
        return f"{value}%"
    return "unknown"

def extract_title_from_citation(citation):
    # Primary regex pattern to capture titles
    match = re.search(
        r'\(\d{4}[a-z]*\)\.\s+(.+?)(?=\.\s|\.$|\s\[|Presented|Paper|Report|Unpublished|Working Paper|Author|Consortium|Evaluation|Study Center|Web Version|conference|Working Group|meeting)',
        citation
    )
    if match:
        return match.group(1).strip()

    # Specific case fallbacks for problematic entries
    specific_patterns = [
        (r'Center for Research on Education Outcomes\. \(June 2009\)\. (Multiple choice: Charter school performance in 16 states)', 1),
        (r'Evans, J\. R\.; Sharma, A\.; Jerabek, A\. \(\d{4}, [A-Za-z/]+\)\. (Building assets and reducing risks whole ninth-grade strategy reduces coursework failure for students of color)', 1),
        (r'Mingle, L\. A\.; Khachatryan, G\.; Kostyuk, V\. \(in press\)\. (Blended learning as a tool for international transfer of instructional practices)', 1),
        (r'UCLA Center X\. \(N\.D\)\. (Report to NETWORK Steering Committee and the USDOE Office of Innovation and Improvement as part of the Investing in Innovation \(i3\) Grant Program Evaluation: Analysis and Summary \(Five Year\))', 1),
        (r'Garcia, J\. L\.; Heckman, J\. J\.; Leaf, D\. E\.; Prados, M\. J\. \((December 2016)\)\. (The life cycle benefits of an influential early childhood program)', 2),
        (r'Cheng, I\.; Ainsworth, A\.; Appelrouth, S\.; Xie, J\.; Moreno, C\. \((2018, April 14)\)\. (Equipping and empowering 8th grade mathematics teachers to create dynamic learning activities promoting conceptual understanding)', 2),
    ]

    for pattern, group in specific_patterns:
        match = re.search(pattern, citation)
        if match:
            return match.group(group).strip()

    # Fallback: Attempt to capture titles from complex cases
    match_fallback = re.search(
        r'\.\s+(.+?)(?=\.\s|\.$|\s\[|Presented|Paper|Report|Unpublished|Working Paper|Author|Consortium|Evaluation|Study Center|Web Version|conference|Working Group|meeting)',
        citation
    )
    if match_fallback:
        return match_fallback.group(1).strip()

    # Final fallback: If all else fails, return "Title Not Found"
    return "Title Not Found"

def check_both(prefix_i, prefix_s, key, row):
    """Check both intervention and study data, prioritizing intervention if both are present."""
    if pd.notnull(row.get(f'{prefix_i}_{key}')):
        return row[f'{prefix_i}_{key}']
    elif pd.notnull(row.get(f'{prefix_s}_{key}')):
        return row[f'{prefix_s}_{key}']
    return None

def aggregate_context_extended(row):
    context_parts = []

    # Delivery Method
    delivery_methods = []
    for method in ['Individual', 'School', 'Small_Group', 'Whole_Class']:
        if check_both('i_Delivery_Method', 's_Delivery_Method', method, row) == 1:
            delivery_methods.append(f"{method.replace('_', ' ')}")
    if delivery_methods:
        context_parts.append(f"Delivery Method: {', '.join(delivery_methods)}")

    # Class Type
    class_types = ['General', 'Inclusion']
    class_type_parts = []
    for class_type in class_types:
        if check_both('i_Class_type', 's_Class_type', class_type, row) == 1:
            class_type_parts.append(class_type)
    if class_type_parts:
        context_parts.append(f"Class Type: {', '.join(class_type_parts)}")

    # Gender
    gender_parts = []
    female_percentage = check_both('i_Gender', 's_Gender', 'Female', row)
    if female_percentage is not None:
        gender_parts.append(f"Female ({format_percentage(female_percentage)})")
    male_percentage = check_both('i_Gender', 's_Gender', 'Male', row)
    if male_percentage is not None:
        gender_parts.append(f"Male ({format_percentage(male_percentage)})")
    if gender_parts:
        context_parts.append(f"Gender: {', '.join(gender_parts)}")

    # Grade
    grades = ['K', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', 'PreK', 'PS']
    grade_parts = [grade for grade in grades if check_both('i_Grade', 's_Grade', grade, row) == 1]
    if grade_parts:
        context_parts.append(f"Grade: {', '.join(grade_parts)}")

    # Race
    races = ['Asian', 'Black', 'Native_American', 'Pacific_Islander', 'White', 'Other']
    race_parts = []
    for race in races:
        race_key = race.replace(' ', '_')
        race_percentage = check_both('i_Race', 's_Race', race_key, row)
        if race_percentage is not None:
            race_parts.append(f"{race.replace('_', ' ')} ({format_percentage(race_percentage)})")
    if race_parts:
        context_parts.append(f"Race: {', '.join(race_parts)}")

    # Ethnicity (Hispanic)
    hispanic_percentage = check_both('i_Ethnicity', 's_Ethnicity', 'Hispanic', row)
    if hispanic_percentage is not None:
        context_parts.append(f"Ethnicity: Hispanic ({format_percentage(hispanic_percentage)})")

    # Demographics
    demographics = {'ELL': 'English Language Learners', 'FRPL': 'Free/Reduced Price Lunch', 'International': 'International Students'}
    demographics_parts = []
    for demo_key, demo_value in demographics.items():
        demo_data = check_both('i_Demographics_Sample', 's_Demographics_Sample', demo_key, row)
        if demo_data is not None:
            percentage = format_percentage(demo_data)
            demographics_parts.append(f"{demo_value} ({percentage})")
    if demographics_parts:
        context_parts.append(f"Demographics: {', '.join(demographics_parts)}")

    # Region and States
    regions = ['Midwest', 'Northeast', 'South', 'West']
    region_parts = [region for region in regions if check_both('i_Region_State', 's_Region_State', region, row) == 1]

    states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New_Hampshire', 'New_Jersey', 'New_Mexico', 'New_York', 'North_Carolina', 'North_Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode_Island', 'South_Carolina', 'South_Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West_Virginia', 'Wisconsin', 'Wyoming']
    state_parts = [state.replace('_', ' ') for state in states if check_both('i_Region_State', 's_Region_State', state, row) == 1]

    if region_parts:
        context_parts.append(f"Region: {', '.join(region_parts)}")
    if state_parts:
        context_parts.append(f"States: {', '.join(state_parts)}")

    # Disability
    disabilities = {'ASD': 'Autism Spectrum Disorder', 'IEP': 'Individualized Education Programs'}
    disability_parts = []
    for code, description in disabilities.items():
        disability_percentage = check_both('i_Disability', 's_Disability', code, row)
        if disability_percentage is not None:
            percentage = format_percentage(disability_percentage)
            disability_parts.append(f"{description} ({percentage})")
    if disability_parts:
        context_parts.append(f"Disability: {', '.join(disability_parts)}")

    # School Type
    school_types = ['Charter', 'Parochial', 'Private', 'Public']
    for school_type in school_types:
        if check_both('i_School_type', 's_School_type', school_type, row) == 1:
            context_parts.append(f"School Type: {school_type}")

    # Urbanicity
    urbanicities = ['Rural', 'Suburban', 'Urban']
    urbanicity_parts = [urbanicity for urbanicity in urbanicities if row.get(f'i_Urbanicity_{urbanicity}', 0) == 1 or row.get(f's_Urbanicity_{urbanicity}', 0) == 1]
    if urbanicity_parts:
        context_parts.append(f"Urbanicity: {', '.join(urbanicity_parts)}")


    # Topic
    topics = ['Literacy', 'STEM', 'College_Readiness_And_Completion', 'Social_Emotional_Learning_And_Behavior', 'SWD', 'ELL', 'Teachers_And_School_Leaders', 'School_Choice', 'Early_Childhood', 'K_to_12th_Grade', 'High_School_Completion', 'Postsecondary']
    topic_parts = [topic.replace('_', ' ') for topic in topics if check_both('i_Topic', 's_Topic', topic, row) == 1]
    if topic_parts:
        context_parts.append(f"Topic: {', '.join(topic_parts)}")

    return '; '.join(context_parts)

def transform_wwc_data(df):
    transformed_entries = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Transforming data"):
        title = extract_title_from_citation(row.get('s_Citation', ''))

        program_type = check_both('i', 's', 'Program_Type', row)
        program_type = program_type if pd.notnull(program_type) else "Unknown"

        originalID = check_both('i', 's', 'InterventionID', row) or check_both('i', 's', 'ProductID', row)
        name = check_both('i', 'f', 'Intervention_Name', row) or check_both('i', 's', 'Product_Name', row)

        eval_sourceURL = "https://ies.ed.gov/ncee/wwc" + (row.get('s_Study_Page_URL', '') or '')
        int_sourceURL = "https://ies.ed.gov/ncee/wwc" + (check_both('i', 's', 'Intervention_Page_URL', row) or '')

        # Generating the context string for each evaluation using aggregate_context_extended
        evaluation_context = aggregate_context_extended(row)

        evaluation = {
            'originalID': row.get('s_StudyID', ''),
            'name': title,
            'reference': row.get('s_Citation', ''),
            'type': row.get('s_Purpose_of_Review', ''),
            'date': row.get('s_Publication_Date', ''),
            'sourceURL': eval_sourceURL,
            'ericID': row.get('s_ERICId', ''),
            'context': evaluation_context,
            'WWC': True
        }

        intervention = {
            'originalID': originalID if originalID else '',
            'name': name if name else '',
            'sourceURL': int_sourceURL,
            'type': program_type
        }

        measure = {
            'originalID': row.get('f_Outcome_MeasureID', ''),
            'name': row.get('f_Outcome_Measure', '')
        }

        outcome_domain = row.get('f_Outcome_Domain', row.get('i_Outcome_Domain', ''))
        outcome = {
            'name': outcome_domain
        }

        effectiveness_rating = map_effectiveness(row.get('i_Effectiveness_Rating', "Unknown"))
        study_rating = map_evidence_quality(row.get('s_Study_Rating', "Unknown"))
        period = row.get('f_Period') if pd.notnull(row.get('f_Period')) else "unknown"

        relationship_properties = {
            'effectiveness': effectiveness_rating,
            'evidenceQuality': study_rating,
            'period': period
        }

        essa_tier = row.get('f_ESSA_Rating', None)
        if pd.notnull(essa_tier):
            relationship_properties['essaTier'] = essa_tier

        relationships = []
        if originalID and row.get('s_StudyID'):
            relationships.append({
                'from': originalID,
                'to': row.get('s_StudyID', ''),
                'type': 'EVALUATED_BY'
            })

        if originalID and outcome_domain:
            relationships.append({
                'from': originalID,
                'to': outcome_domain,
                'type': 'LEADS_TO',
                'properties': relationship_properties
            })

        if row.get('s_StudyID') and row.get('f_Outcome_MeasureID'):
            relationships.append({
                'from': row.get('s_StudyID', ''),
                'to': row.get('f_Outcome_MeasureID', ''),
                'type': 'UTILIZES'
            })

        if row.get('f_Outcome_MeasureID') and outcome_domain:
            relationships.append({
                'from': row.get('f_Outcome_MeasureID', ''),
                'to': outcome_domain,
                'type': 'MEASURES'
            })

        # Create a single JSON object for the current entry
        entry = {
            'evaluation': evaluation,
            'intervention': intervention,
            'outcome': outcome,
            'measure': measure,
            'relationships': relationships
        }

        # Add the entry to the transformed entries list
        transformed_entries.append(entry)

    return transformed_entries

# Transform the data
transformed_wwc = transform_wwc_data(df_wwc)
entry_index = 100  # Update this to try with different entry indices
print(f"Serialized data for entry {entry_index}:")
pprint(transformed_wwc[entry_index])


Transforming data:   0%|          | 0/11901 [00:00<?, ?it/s]

Serialized data for entry 100:
{'evaluation': {'WWC': True,
                'context': 'Delivery Method: Small Group, Whole Class; Gender: '
                           'Female (47.78%), Male (52.21%); Race: Asian '
                           '(0.0%), Black (56.79%), Pacific Islander (0.0%), '
                           'White (12.87%); Ethnicity: Hispanic (35.65%); '
                           'Demographics: English Language Learners (100.0%); '
                           'Region: South, West; States: California, Florida; '
                           'Disability: Individualized Education Programs '
                           '(36.0%); School Type: Public; Urbanicity: Urban; '
                           'Topic: Literacy, STEM, Early Childhood',
                'date': '2008',
                'ericID': 'ED502153',
                'name': 'Effects of Preschool Curriculum Programs on School '
                        'Readiness',
                'originalID': '29250',
                'refer

In [ ]:
entry_index = 9468  # Update this to try with different entry indices
print(f"Serialized data for entry {entry_index}:")
pprint(transformed_wwc[entry_index])

Serialized data for entry 9468:
{'evaluation': {'WWC': True,
                'context': 'Delivery Method: Individual, Small Group; Gender: '
                           'Female (23.5%), Male (76.5%); Grade: 1, 2; Race: '
                           'Asian (11.8%), Black (5.9%), White (76.5%), Other '
                           '(5.8%); Ethnicity: Hispanic (11.8%); Region: '
                           'South; States: North Carolina; Disability: Autism '
                           'Spectrum Disorder (100.0%); Urbanicity: Urban; '
                           'Topic: Social Emotional Learning And Behavior, '
                           'SWD, K to 12th Grade',
                'date': '2019',
                'ericID': 'ED607066',
                'name': 'Efficacy Study of a Social Communication and '
                        'Self-Regulation Intervention for School-Age Children '
                        'with Autism Spectrum Disorder: A Randomized '
                        'Controlled Trial',
   

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm.notebook import tqdm

# Setup Neo4j connection
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def load_data_to_neo4j(transformed_entries, driver, batch_size=1000):
    def insert_nodes(tx, nodes):
        for label in ["Evaluation", "Intervention", "Measure"]:
            query = f"""
            UNWIND $nodes as node
            MERGE (n:{label} {{originalID: node.originalID}})
            ON CREATE SET n += node.properties
            ON MATCH SET n += node.properties
            """
            filtered_nodes = [node for node in nodes if node['label'] == label]
            if filtered_nodes:
                tx.run(query, nodes=filtered_nodes)

    def insert_outcome_nodes(tx, outcomes):
        query = """
        UNWIND $outcomes as outcome
        MERGE (o:Outcome {name: outcome.name})
        ON CREATE SET o += outcome.properties
        ON MATCH SET o += outcome.properties
        """
        tx.run(query, outcomes=outcomes)

    def insert_relationships(tx, relationships, rel_type):
        if rel_type == 'LEADS_TO':
            query = """
            UNWIND $relationships as rel
            MATCH (fromNode:Intervention {originalID: rel.fromID})
            MATCH (toNode:Outcome {name: rel.toID})
            MERGE (fromNode)-[r:LEADS_TO]->(toNode)
            SET r += rel.properties
            """
        elif rel_type == 'MEASURES':
            query = """
            UNWIND $relationships as rel
            MATCH (fromNode:Measure {originalID: rel.fromID})
            MATCH (toNode:Outcome {name: rel.toID})
            MERGE (fromNode)-[r:MEASURES]->(toNode)
            SET r += rel.properties
            """
        elif rel_type == 'EVALUATED_BY':
            query = """
            UNWIND $relationships as rel
            MATCH (fromNode:Intervention {originalID: rel.fromID})
            MATCH (toNode:Evaluation {originalID: rel.toID})
            MERGE (fromNode)-[r:EVALUATED_BY]->(toNode)
            SET r += rel.properties
            """
        elif rel_type == 'UTILIZES':
            query = """
            UNWIND $relationships as rel
            MATCH (fromNode:Evaluation {originalID: rel.fromID})
            MATCH (toNode:Measure {originalID: rel.toID})
            MERGE (fromNode)-[r:UTILIZES]->(toNode)
            SET r += rel.properties
            """
        tx.run(query, relationships=relationships)

    nodes = []
    outcomes = []
    relationships = {
        'EVALUATED_BY': [],
        'LEADS_TO': [],
        'MEASURES': [],
        'UTILIZES': []
    }

    for entry in transformed_entries:
        # Prepare nodes
        if 'evaluation' in entry and pd.notnull(entry['evaluation']['originalID']):
            nodes.append({
                'originalID': entry['evaluation']['originalID'],
                'label': "Evaluation",
                'properties': entry['evaluation']
            })

        if 'intervention' in entry and pd.notnull(entry['intervention']['originalID']):
            nodes.append({
                'originalID': entry['intervention']['originalID'],
                'label': "Intervention",
                'properties': entry['intervention']
            })

        if 'measure' in entry and pd.notnull(entry['measure']['originalID']):
            nodes.append({
                'originalID': entry['measure']['originalID'],
                'label': "Measure",
                'properties': entry['measure']
            })

        if 'outcome' in entry and pd.notnull(entry['outcome']['name']):
            outcomes.append({
                'name': entry['outcome']['name'],
                'properties': entry['outcome']
            })

        # Prepare relationships
        for relationship in entry['relationships']:
            if pd.notnull(relationship['from']) and pd.notnull(relationship['to']):
                relationships[relationship['type']].append({
                    'fromID': relationship['from'],
                    'toID': relationship['to'],
                    'properties': relationship.get('properties', {})
                })

    with driver.session() as session:
        for i in tqdm(range(0, len(nodes), batch_size), desc="Loading nodes to Neo4j"):
            batch_nodes = nodes[i:i + batch_size]
            session.execute_write(insert_nodes, batch_nodes)

        for i in tqdm(range(0, len(outcomes), batch_size), desc="Loading outcomes to Neo4j"):
            batch_outcomes = outcomes[i:i + batch_size]
            session.execute_write(insert_outcome_nodes, batch_outcomes)

        for rel_type, rels in relationships.items():
            for i in tqdm(range(0, len(rels), batch_size), desc=f"Loading {rel_type} relationships to Neo4j"):
                batch_relationships = rels[i:i + batch_size]
                session.execute_write(insert_relationships, batch_relationships, rel_type)

# Load the transformed data into Neo4j
load_data_to_neo4j(transformed_wwc, driver)

driver.close()


Loading nodes to Neo4j:   0%|          | 0/36 [00:00<?, ?it/s]

Loading outcomes to Neo4j:   0%|          | 0/12 [00:00<?, ?it/s]

Loading EVALUATED_BY relationships to Neo4j:   0%|          | 0/4 [00:00<?, ?it/s]

Loading LEADS_TO relationships to Neo4j:   0%|          | 0/4 [00:00<?, ?it/s]

Loading MEASURES relationships to Neo4j:   0%|          | 0/12 [00:00<?, ?it/s]

Loading UTILIZES relationships to Neo4j:   0%|          | 0/12 [00:00<?, ?it/s]

Need to manually update one article's title (Jemmott III, J) and remove the ericIDs that have NaN